In [1]:
import random
import numpy as np
from collections import deque
import pandas as pd 
import time
import networkx as nx
import copy
from gurobipy import *
%run PL.ipynb
process

<function __main__.process(TVR, alphaPM, alphaPF, aplhaSlack, DifMaxPal, L, NombrePala, ND, NP, PM0, F, PF0, PDisp, TC, CP, TD, CD, DDisp, DMPP, cS1, cS2, cS3, cS4, cS5, NC, t_global, LocPala, NombreDestino, Ley, LeyPromedio, NombreFases)>

In [2]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

# Parámetros

In [3]:
#Horizonte de simulacion (Turnos)
espacios = 5 #Espacios de aculatamiento por stock
Turnos = 1
Reloj = 0
tiempo_aculatamiento = 70/60
AlmuerzoTA = 60 #Duracion almuerzo turno A
Almuerzo1TB = 40 #Duracion colacion 1 turno B
Almuerzo2TB = 20 #Duracion colacion 2 turno B
nodo_casino = "BAlpaca" #Nombre del nodo donde esta ubicado el casino
nodo_chancador = "CH1" #Nombre del nodo donde esta ubicado el chancador
comienzo_camion = "CS03" #Nombre del lugar donde parten los camiones
size_camion = 300
Extraccion_Mineral = 303
Extraccion_Esteril = 293
Nombres_Fases = ['Fase1', 'Fase2', 'Fase3', 'Fase4']
LeyPromedio = 1.1
Nodos_Fase = {'Fase1': {1:['PA21'], 0: ['PA15']}, 'Fase2': {1: ['PA13'], 0: ['PA12']}}
Nodos_Fase['Fase3'] = {1:[], 0:[]}
Nodos_Fase['Fase4'] = {1:[], 0:[]}

D_Plan_Esteril = {'Fase1': 45000, 'Fase2': 30000, 'Fase3': 0, 'Fase4': 0}
Plan_Esteril_Actualizado = copy.deepcopy(D_Plan_Esteril)

def getNodos(Nodos_Fase):
    Nodos_Mineral = []
    Nodos_Esteril = []
    
    for fase in Nodos_Fase.keys():
        Nodos_Mineral += Nodos_Fase[fase][1]
        Nodos_Esteril += Nodos_Fase[fase][0]
        
    Polvorazos = Nodos_Mineral + Nodos_Esteril
    return(Nodos_Mineral, Nodos_Esteril, Polvorazos)

def modificar_nodo(Nodos_Fase, nodo, clase):#Cambia un nodo que era de mineral a esteril o viceversa
    for fase in Nodos_Fase.keys():
        if nodo in Nodos_Fase[fase][0] and clase != 0:
            Nodos_Fase[fase][0].remove(nodo)
            Nodos_Fase[fase][1].append(nodo)
            return(True)
        if nodo in Nodos_Fase[fase][1] and clase != 1:
            Nodos_Fase[fase][1].remove(nodo)
            Nodos_Fase[fase][0].append(nodo)
            return(True)
           
Nodos_Mineral, Nodos_Esteril, Polvorazos = getNodos(Nodos_Fase)
LeyCobrePromedio=[1.32515020622562, 0.0]



# Funciones

In [4]:
def Pi_Real(toneladas_sacadas, reloj):#Toneladas sacadas la ultima hora
    Pi_real = {}
    for item in toneladas_sacadas[::-1]:
        if item[0] >= reloj-60:
            try:
                Pi_real[item[4]] += item[2]
            except:
                Pi_real[item[4]] = item[2]
    return(Pi_real)#Retorna un diccionario con el TPH real por camino

def Aj(Pala, reloj):
    camiones_asignados = Pala.camiones_asignados
    aj = 0
    for item in camiones_asignados[::-1]:
        if item[0] >= reloj-60:
            aj += Pala.capacidad
    return(aj)

def TPH_Teorico(TPH_PL, Pala):#Rj
    idPala = Pala.ID
    Rj = 0
    for key in TPH_PL[idPala].keys():
        Rj += TPH_PL[idPala][key]
    return(Rj)

def Truck_idle(D_Trucks, Pala, reloj, TdV):#Llegada = Reloj + Tiempo de viaje
    camiones = 0
    Llegada = reloj + TdV
    tasa_atencion = Pala.tasa_atencion
    for camion in D_Camiones.keys():
        #Si el camion va viajando hacia la pala
        truck = D_Camiones[camion]
        if truck.estado == "Viajando" and truck.destino == Pala.ID and truck.hora_siguiente <= Llegada:
            camiones += 1
        #Si el camion esta descargando o aculatandose, en la fila o viajando hacia un chancador, stock o botadero  
    camiones += len(Pala.fila) 
    camiones += len(Pala.orden_llegada)
    camiones = max(0, camiones - tasa_atencion*TdV)
    return(camiones/tasa_atencion)
      
def camiones_viajando(D_Camiones, Pala, D_Entidades):#Se utiliza esta funcion en caida de pala/chancador
    #Identifica los camiones viajando a la pala y los que van a descargr
    output = []
    a_descarga = []
    for camion in D_Camiones.keys():
        #Si el camion va viajando hacia la pala
        truck = D_Camiones[camion]
        #print(truck, truck.estado, truck.origen, truck.destino)
        if truck.estado != "Almorzando":
            try:
                clase_destino = D_Entidades[truck.destino].__class__.__name__
                if truck.estado == "Viajando" and truck.destino == Pala.ID:
                    output.append(truck) 
                #Si el camion esta descargando o aculatandose, en la fila o viajando hacia un chancador, stock o botadero  
                #elif truck.estado == "Descargando" or ((truck.estado in ["En fila", "Aculatandose", "Viajando"] and \
                #                                                   clase_destino in ["Chancador", "Stock"])):
                #    a_descarga.append(D_Camiones[camion])
            except:
                pass

    output.sort(key=lambda x: x.hora_siguiente, reverse=False)
    return(output)#output: camiones viajando a Pala, a_descarga = camiones en camino a descarga

def camiones_asignables(D_Camiones, D_Entidades):#Se utiliza esta funcion para alimentar el dinamico en situacion normal
    output = []
    for camion in D_Camiones.keys():
        truck = D_Camiones[camion]
        #print(truck, truck.estado, truck.origen, truck.destino)
        if truck.estado != "Almorzando":
            try:
                clase_destino = D_Entidades[truck.destino].__class__.__name__
                if truck.estado == "Descargando" or ((truck.estado in ["En fila", "Aculatandose", "Viajando"] and \
                                                                   clase_destino in ["Chancador", "Stock"])):
                    output.append(truck)
            except:
                pass
    return(output)

def getNINT(truck, D_Entidades):
    try:
        NI = D_Entidades[truck.origen].ubicacion  #Nodo de donde viene viajando el camion
        NT = D_Entidades[truck.destino].ubicacion #Nodo hacia donde va viajando el camion

    except:
        if truck.origen not in D_Entidades.keys():
            NI = truck.origen 
        else:
            NI = D_Entidades[truck.origen].ubicacion 

        if truck.destino not in D_Entidades.keys():
            NT = truck.destino
        else:
            NT = D_Entidades[truck.destino].ubicacion
    return(NI, NT)

def Hora_Almuerzo(D_Entidades, Entidad, Reloj, TurnoA, TurnoB, MIT, MTV, AlmuerzoTA, Almuerzo1TB, Almuerzo2TB, nodo_casino):
    if (TurnoA and 0 <= Entidad.ID and Entidad.ID <= 40 and Reloj >= (4.5*60 + MIT) and Entidad.almuerzo == 0):
        Destino = D_Entidades[Entidad.destino]
        if len(Destino.fila) > 0:
            #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
            i = Destino.espacios_aculatamiento.index(Entidad)
            Siguiente = Destino.fila.popleft()
            Destino.espacios_aculatamiento[i] = Siguiente
            Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
        else:
            i = Destino.espacios_aculatamiento.index(Entidad)
            Destino.espacios_aculatamiento[i] = 0
        TV = MTV[D_Entidades[Entidad.destino].ubicacion][nodo_casino][0]
        Entidad.comenzar_almuerzo(nodo_casino, Reloj, TV, AlmuerzoTA, 1)

    if (TurnoA and Entidad.ID > 40 and Reloj >= (6*60 + MIT) and Entidad.almuerzo ==0):
        Destino = D_Entidades[Entidad.destino]
        if len(Destino.fila) > 0:
            #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
            i = Destino.espacios_aculatamiento.index(Entidad)
            Siguiente = Destino.fila.popleft()
            Destino.espacios_aculatamiento[i] = Siguiente
            Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
        else:
            i = Destino.espacios_aculatamiento.index(Entidad)
            Destino.espacios_aculatamiento[i] = 0
        TV = MTV[D_Entidades[Entidad.destino].ubicacion][nodo_casino][0]
        Entidad.comenzar_almuerzo(nodo_casino, Reloj, TV, AlmuerzoTA, 1)

    #Primera colacion turno B
    if (TurnoB and 0 <= Entidad.ID and Entidad.ID <= 27 and Reloj >= (5*60 + MIT) and Entidad.almuerzo == 0):
        Destino = D_Entidades[Entidad.destino]
        if len(Destino.fila) > 0:
            i = Destino.espacios_aculatamiento.index(Entidad)
            Siguiente = Destino.fila.popleft()
            Destino.espacios_aculatamiento[i] = Siguiente
            Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
        else:
            i = Destino.espacios_aculatamiento.index(Entidad)
            Destino.espacios_aculatamiento[i] = 0

        Entidad.comenzar_almuerzo(Entidad.destino, Reloj, 0, Almuerzo1TB, 1)

    if (TurnoB and 28 <= Entidad.ID and Entidad.ID <= 54 and Reloj >= (5.5*60 + MIT)and Entidad.almuerzo == 0):
        Destino = D_Entidades[Entidad.destino]
        if len(Destino.fila) > 0:
            #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
            i = Destino.espacios_aculatamiento.index(Entidad)
            Siguiente = Destino.fila.popleft()
            Destino.espacios_aculatamiento[i] = Siguiente
            Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
        else:
            i = Destino.espacios_aculatamiento.index(Entidad)
            Destino.espacios_aculatamiento[i] = 0
        Entidad.comenzar_almuerzo(Entidad.destino, Reloj, 0, Almuerzo1TB, 1)

    if (TurnoB and 55 <= Entidad.ID and Reloj >= (4.5*60 + MIT) and Entidad.almuerzo == 0):
        Destino = D_Entidades[Entidad.destino]
        if len(Destino.fila) > 0:
            #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
            i = Destino.espacios_aculatamiento.index(Entidad)
            Siguiente = Destino.fila.popleft()
            Destino.espacios_aculatamiento[i] = Siguiente
            Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
        else:
            i = Destino.espacios_aculatamiento.index(Entidad)
            Destino.espacios_aculatamiento[i] = 0
        Entidad.comenzar_almuerzo(Entidad.destino, Reloj, 0, Almuerzo1TB, 1)

    #Segunda colacion turno B
    if (TurnoB and 0 <= Entidad.ID and Entidad.ID <= 45 and Reloj >= (9*60 + MIT) and Entidad.almuerzo == 1):
        Destino = D_Entidades[Entidad.destino]
        if len(Destino.fila) > 0:
            #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
            i = Destino.espacios_aculatamiento.index(Entidad)
            Siguiente = Destino.fila.popleft()
            Destino.espacios_aculatamiento[i] = Siguiente
            Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
        else:
            i = Destino.espacios_aculatamiento.index(Entidad)
            Destino.espacios_aculatamiento[i] = 0
        Entidad.comenzar_almuerzo(Entidad.destino, Reloj, 0, Almuerzo2TB, 2)

    if (TurnoB and 46 <= Entidad.ID and Reloj >= (9.5*60 + MIT) and Entidad.almuerzo == 1):
        Destino = D_Entidades[Entidad.destino]
        if len(Destino.fila) > 0:
            #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
            i = Destino.espacios_aculatamiento.index(Entidad)
            Siguiente = Destino.fila.popleft()
            Destino.espacios_aculatamiento[i] = Siguiente
            Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
        else:
            i = Destino.espacios_aculatamiento.index(Entidad)
            Destino.espacios_aculatamiento[i] = 0
        Entidad.comenzar_almuerzo(Entidad.destino, Reloj, 0, Almuerzo2TB, 2)
        

In [5]:
#Se leen las caracteristicas de las palas
dfPalas = pd.read_excel('Informacion Palas.xlsx',  1)
InfoPalas = list(zip(dfPalas[dfPalas.columns[0]].values, dfPalas[dfPalas.columns[1]].values, dfPalas[dfPalas.columns[2]].values, \
                            dfPalas[dfPalas.columns[3]].values, dfPalas[dfPalas.columns[4]].values, \
                            dfPalas[dfPalas.columns[5]].values, dfPalas[dfPalas.columns[6]].values, \
                            dfPalas[dfPalas.columns[7]].values, dfPalas[dfPalas.columns[8]].values))

        
Plan_Mineral_Actualizado = {} #Por pala
for pala in InfoPalas:
    Plan_Mineral_Actualizado[pala[0]] = random.uniform(16000, 18000)
Plan_Mineral = copy.deepcopy(Plan_Mineral_Actualizado)
#Número de camiones disponibles
total_camiones = 40

#Lista de stocks/botaderos
#ID, espacios, ubicacion, tiempo_atencion ID=ubicacion?
SoB = ['Botadero1','Botadero2','Botadero3','SPGolondrinaSME','SPGaviotaSAL','SPSulFeDulce','SP1','SP2','SP3','SP4']
L_Botaderos = ['SPSulFeDulce']
L_Stocks = ['SPGolondrinaSME','SPGaviotaSAL']

tiempo_atencion_stock = 70/60 

#Se lee la informacion del chancador
dfChancador = pd.read_excel('Informacion Palas.xlsx',  0)
InfoChancador = list(zip(dfChancador[dfChancador.columns[0]].values, dfChancador[dfChancador.columns[1]].values, dfChancador[dfChancador.columns[2]].values, \
                            dfChancador[dfChancador.columns[3]].values, dfChancador[dfChancador.columns[4]].values, \
                            dfChancador[dfChancador.columns[5]].values, dfChancador[dfChancador.columns[6]].values, \
                            dfChancador[dfChancador.columns[7]].values, dfChancador[dfChancador.columns[8]].values))


print("Nodos de Mineral: {}".format(Nodos_Mineral))
print("Nodos de Esteril: {}".format(Nodos_Esteril))
print("Polvorazos: {}".format(Polvorazos))
print("Plan de Esteril: {} ".format(D_Plan_Esteril))
print("Plan de Mineral: {}".format(Plan_Mineral))

Nodos de Mineral: ['PA21', 'PA13']
Nodos de Esteril: ['PA15', 'PA12']
Polvorazos: ['PA21', 'PA13', 'PA15', 'PA12']
Plan de Esteril: {'Fase1': 45000, 'Fase2': 30000, 'Fase3': 0, 'Fase4': 0} 
Plan de Mineral: {'PAA': 16113.785889051493, 'PAB': 17364.67613973196, 'PAC': 16231.40752746677, 'PAD': 16355.530631677593}


# Creación de grafo

In [6]:
dfGrafo= pd.read_excel('Velocidad cargado.xls')
dfGrafo.columns
Arcos = []
Nodos_grafo = list(dfGrafo.columns)

for origen in Nodos_grafo:
    i = 0
    for destino in Nodos_grafo:
        if dfGrafo[origen][i] != 0:
            tiempo = random.uniform(1, 5)
            Arcos.append([origen, destino, tiempo, tiempo+random.uniform(1,4), 1])
        i += 1
#print(Arcos)       

# Dinamico

In [7]:
def dinamico(destinos, pala):
    return (random.choice(pala), random.choice(destinos))

def dinamico_dispatch(caminosMinimos, Camion, L_TrucksOG, PalasOG, MTV, reloj, TPH_PL, D_EntidadesOG, nodo_casino):
    #Camion es el que llama al dinamico.
    #D_Trucks es la lista de camiones que podrian ser asignados (viajando a la pala o viajando a un stock/cs03)
    #Palas es el diccionario de palas disponibles
    #MTV es la matriz de tiempos de viaje
    #reloj en minutos que se llama al dinamico
    #TPH_PL los flujos resultantes del PL
    #caminosMinimos es el diccionario con los shortest path entre dos puntos para viajes vacios
    L_Trucks = copy.deepcopy(L_TrucksOG)
    Palas = copy.deepcopy(PalasOG)
    D_Entidades = copy.deepcopy(D_EntidadesOG)
    '-------------------------------------------------------------------------------------------------------------'
    need_time = []
    Total_rate = 0
    for key1 in Palas.keys():
        Pala = Palas[key1]
        #print(Pala)
        if Pala.estado == "Funcionando":
            aj = Aj(Pala, reloj) #Flujo real de camiones que ha sido asignado a la pala en la ultima hora
            Pi = Pi_Real(Pala.toneladas_sacadas, reloj) #Flujo real que ha salido por el path i 
            Lj = (int(reloj/720)*720-Pala.hora_ultima_asignacion)/60#Hora en que fue asignado el ultimo camion a la pala
            Rj = TPH_Teorico(TPH_PL, Pala) #TPH teorico
            Total_rate += Rj
            for key2 in TPH_PL[Pala.ID].keys():
                
                Pi_teorico = TPH_PL[Pala.ID][key2] #Flujo teorico que deberia haber salido por el path i
                
                try:
                    Pi_real = Pi[key2]
                except:
                    Pi_real = 0
                    
                if Pi_teorico != 0:
                    needtimei = Lj + ((Pi_teorico/Rj)*(aj-Rj))/(Pi_real+1) 
                    need_time.append([Pala, key2, needtimei])
              
    need_time = sorted(need_time, key=lambda x: x[2]) #Se ordena de menor a mayor 
    best_truck = None
    #Calcular el lost tons para cada camion y hacer el match iterativo recalculando los needtime (actualizando Lj, etc.)
    Required_trucks = 81*300
    #L_Trucks_Copia = L_Trucks.copy()
    if Camion not in L_Trucks:
        L_Trucks.append(Camion)
    #print(L_Trucks_Copia)

    while True:
        best_truck = None
        min_tones = 99999999999999999
        for path in need_time:
            Pala = path[0]
            for truck in L_Trucks:
                #print(truck)
                #Hay que ver que pasa con los camiones que estan llendo a descargar
                try:
                    NI = D_Entidades[truck.origen].ubicacion  #Nodo de donde viene viajando el camion
                    NT = D_Entidades[truck.destino].ubicacion #Nodo hacia donde va viajando el camion
                
                except:
                    if truck.origen not in D_Entidades.keys():
                        NI = truck.origen  
                    else:
                        NI = D_Entidades[truck.origen].ubicacion 
                        
                    if truck.destino not in D_Entidades.keys():
                        NT = truck.destino
                    else:
                        NT = D_Entidades[truck.destino].ubicacion
                      
                nodo_inicio = nuevoNodoInicioTrasCaida(caminosMinimos, NI, NT, reloj-truck.hora_salida)
                
                if truck.cargado == 1:
                    TVPala = (MTV[nodo_inicio][NT][1] + MTV[NT][Pala.ubicacion][0])/60
                elif truck.cargado == 0:
                    TVPala =  MTV[nodo_inicio][Pala.ubicacion][truck.cargado]/60
                    
                idle_time = Truck_idle(L_Trucks, Pala, reloj, TVPala)/60
                Shovel_Rate = TPH_Teorico(TPH_PL, Pala)
                Shovel_Idle = 0
                if Pala.fila == 0:
                    Shovel_Idle = TVPala
                
                lost_tones = (idle_time + TVPala)*(truck.size*Total_rate)/Required_trucks + (Shovel_Idle * Shovel_Rate)
                if lost_tones < min_tones:
                    min_tones = lost_tones
                    best_truck = truck
            
            if best_truck == Camion:
                #Pala.hora_ultima_asignacion = reloj
                #Pala.camiones_asignados.append((reloj, camion))
                print((Pala.ID, path[1]))
                return((Pala.ID, path[1]))
            
            else:
                #print(best_truck)
                Pala.hora_ultima_asignacion = reloj
                Pala.camiones_asignados.append((reloj, best_truck))
                L_Trucks.remove(best_truck)
                for elemento in need_time:
                    if elemento[0] == Pala:
                        #print(elemento, Pala)
                        aj = Aj(Pala, reloj) #Flujo real de camiones que ha sido asignado a la pala en la ultima hora
                        try:
                            Pi_real = Pi_Real(Pala.toneladas_sacadas, reloj)[elemento[1]] #Flujo real que ha salido por el path i 
                        except:
                            Pi_real = 0
                        Lj = (int(reloj/720)*720-Pala.hora_ultima_asignacion)/60#Hora en que fue asignado el ultimo camion a la pala
                        Rj = TPH_Teorico(TPH_PL, Pala) #TPH teorico
                        Pi_teorico = TPH_PL[Pala.ID][elemento[1]]
                        needtimei = Lj + ((Pi_teorico/Rj)*(aj-Rj))/(Pi_real+1)
                        #need_time.remove(path)
                        #need_time.append((Pala, path[1], needtimei))
                        elemento[2] = needtimei
                        #print(elemento)
                need_time = sorted(need_time, key=lambda x: x[2])        
                break

    ###Agregar resultado a lista de camiones asignados y modificar la hora de ultima asignacion de la pala
    ##Modificar el par OD del camion y el nuevo destino     

#dinamico_dispatch(caminosMinimosYTiemposAcumulados, ,  camiones_asignables(D_Camiones, D_Entidades), D_Palas, MTV, Reloj, PL, D_Entidades)   
#dinamico_dispatch(caminosMinimosYTiemposAcumulados, Entidad,  camiones_asignables(D_Camiones, D_Entidades), D_Palas, MTV, Reloj, PL, D_Entidades)
#dinamico_dispatch(caminosMinimosYTiemposAcumulados, D_Entidades[0],  camiones_asignables(D_Camiones, D_Entidades), D_Palas, MTV, Reloj, PL, D_Entidades, nodo_casino)
#dinamico_dispatch(caminosMinimosYTiemposAcumulados, Entidad, camiones_asignables(D_Camiones, D_Entidades), D_Palas, MTV, Reloj, PL, D_Entidades, nodo_casino)

# Floyd Warshall

In [8]:
def nuevoNodoInicioTrasCaida(caminosMinimos, NodoInicioActual, NodoTerminoActual, tiempoTranscurrido):
    listaNodos=caminosMinimos[(NodoInicioActual,NodoTerminoActual)][0]
    listaTiemposAcumulados = caminosMinimos[(NodoInicioActual,NodoTerminoActual)][1]
    for i in range(0,len(listaNodos)):
        if tiempoTranscurrido <= listaTiemposAcumulados[i]:
            if i == 0:
                return NodoInicioActual
            else:
                if tiempoTranscurrido <= (listaTiemposAcumulados[i-1]+listaTiemposAcumulados[i])/2:
                    return listaNodos[i-1]
                else:
                    return listaNodos[i]
    return NodoTerminoActual

#INPUTS DISPATCH

#Tabla donde está guardado el grafo (origen,destino,tiempo vacío, tiempo cargado, cerrado { 1,0})
#MTV=[['C-ENTRENAMIENTO', 'NR216', 55, 94, 0], ['CO01', 'NU58', 18, 19, 0], ['CS01', 'NU41', 14, 31, 0], ['CS03', 'NR347', 7, 18, 0], ['CS03', 'NR353', 15, 9, 0], ['ESTAC 4600', 'NR231', 57, 85, 0], ['NAVE01', 'TALLER ROSARIO', 10, 8, 0], ['NAVE02', 'TALLER ROSARIO', 6, 7, 0], ['NAVE03', 'TALLER ROSARIO', 6, 6, 0], ['NAVE04', 'TALLER ROSARIO', 6, 5, 0], ['NAVE05', 'TALLER ROSARIO', 7, 5, 0], ['NAVE06', 'TALLER ROSARIO', 5, 5, 0], ['NAVE07', 'TALLER ROSARIO', 4, 5, 0], ['NAVE08', 'TALLER ROSARIO', 9, 7, 0], ['NAVE09', 'TALLER ROSARIO', 9, 7, 0], ['NAVE10', 'TALLER ROSARIO', 7, 7, 0], ['NAVE11', 'TALLER ROSARIO', 9, 6, 0], ['NAVE12', 'TALLER ROSARIO', 10, 6, 0], ['NAVE13', 'TALLER ROSARIO', 6, 6, 0], ['NAVE14', 'TALLER ROSARIO', 7, 12, 0], ['NAVE15', 'TALLER ROSARIO', 6, 9, 0], ['NAVE16', 'TALLER ROSARIO', 8, 14, 0], ['NAVE17', 'TALLER ROSARIO', 12, 8, 0], ['NAVE18', 'TALLER ROSARIO', 7, 10, 0], ['NR1000', 'NR1047', 22, 39, 0], ['NR1000', 'NR1029', 84, 99, 0], ['NR1000', 'NR1041', 32, 43, 0], ['NR1001', 'NR1020', 38, 41, 0], ['NR1001', 'NR231', 35, 42, 0], ['NR1002', 'NR1031', 26, 22, 0], ['NR1002', 'NR1040', 22, 23, 0], ['NR1003', 'NR1045', 106, 125, 0], ['NR1003', 'NR1029', 30, 26, 0], ['NR1003', 'NR1036', 73, 86, 1], ['NR1004', 'NR1005', 457, 544, 0], ['NR1004', 'NR1006', 31, 25, 0], ['NR1005', 'NR1021', 148, 208, 0], ['NR1005', 'NR1004', 36, 28, 0], ['NR1005', 'NR1037', 68, 71, 0], ['NR1006', 'NR1004', 82, 97, 0], ['NR1006', 'NR1007', 34, 27, 0], ['NR1007', 'NR1006', 272, 324, 0], ['NR1007', 'NR1032', 47, 37, 0], ['NR1008', 'NR1026', 405, 572, 0], ['NR1008', 'NR1021', 38, 30, 1], ['NR1009', 'NR297', 32, 35, 0], ['NR1009', 'NR1020', 98, 120, 0], ['NR1010', 'NR1042', 39, 74, 0], ['NR1062', 'RF10B-4180-1005-1', 78, 36, 0], ['RI10B-INPIT/SBL', 'NR1063', 11, 14, 0], ['NR1010', 'RI10B-INPIT/SME', 8, 12, 0], ['NR1010', 'RI10A-SOBRECARGA/EST', 10, 12, 0], ['NR1010', 'RI10B-INPIT/EST', 7, 9, 0], ['NR1010', 'RF10B-4195-1005-1', 23, 19, 0], ['NR1011', 'NR1013', 35, 42, 0], ['NR1011', 'NR312', 157, 165, 0], ['NR1012', 'NR1023', 80, 64, 0], ['NR1012', 'NR1027', 59, 55, 0], ['NR1013', 'NR1014', 44, 27, 0], ['NR1013', 'NR1011', 35, 36, 0], ['NR1013', 'NR1211', 32, 36, 0], ['NR1014', 'NR1019', 31, 26, 0], ['NR1014', 'NR1013', 70, 100, 0], ['NR1015', 'NR1035', 45, 53, 0], ['NR1015', 'RAMPA CAPELLA', 26, 28, 0], ['NR1016', 'NR1054', 57, 83, 0], ['NR1016', 'NR1052', 26, 32, 1], ['NR1017', 'NR1018', 37, 56, 0], ['NR1017', 'NR1023', 53, 78, 0], ['NR1018', 'NR1017', 48, 60, 0], ['NR1018', 'NR1025', 37, 52, 0], ['NR1019', 'NR1025', 28, 25, 0], ['NR1019', 'NR1014', 17, 19, 0], ['NR1020', 'NR1027', 55, 53, 0], ['NR1020', 'NR1001', 35, 42, 0], ['NR1020', 'NR1009', 56, 41, 0], ['NR1021', 'NR1005', 25, 20, 0], ['NR1021', 'NR1008', 220, 310, 1], ['NR1022', 'NR1040', 35, 37, 0], ['NR1022', 'NR1028', 20, 18, 0], ['NR1023', 'NR1012', 48, 50, 0], ['NR1023', 'NR1017', 71, 86, 0], ['NR1023', 'RAMPA 4600', 61, 48, 0], ['NR1024', 'NR1026', 38, 40, 0], ['NR1024', 'RAMPA 4600', 41, 47, 0], ['NR1024', 'NR1381', 41, 62, 0], ['NR1025', 'NR1018', 32, 26, 0], ['NR1025', 'NR1019', 25, 39, 0], ['NR1026', 'NR1024', 49, 54, 0], ['NR1026', 'NR1008', 47, 37, 0], ['NR1027', 'NR1020', 34, 42, 0], ['NR1027', 'NR1012', 150, 156, 0], ['NR1027', 'NR1382', 63, 76, 0], ['NR1028', 'NR1046', 22, 17, 0], ['NR1028', 'NR1030', 16, 19, 0], ['NR1028', 'NR1022', 48, 45, 0], ['NR1029', 'NR1003', 174, 204, 0], ['NR1029', 'NR1000', 36, 31, 1], ['NR1030', 'NR1061', 61, 62, 0], ['NR1030', 'NR1028', 15, 20, 0], ['NR1031', 'NR1053', 35, 42, 0], ['NR1031', 'NR1002', 21, 20, 0], ['NR1032', 'NR1036', 27, 22, 0], ['NR1032', 'NR1007', 330, 394, 0], ['NR1033', 'NR1050', 10, 26, 0], ['NR1033', 'NR1107', 15, 12, 0], ['NR1034', 'NR915', 18, 17, 1], ['NR1034', 'NR923', 17, 18, 0], ['NR1034', 'NR1043', 22, 16, 0], ['NR1035', 'NR1043', 50, 101, 0], ['NR1035', 'NR1015', 33, 39, 0], ['NR1036', 'NR1032', 69, 97, 0], ['NR1036', 'NR1003', 40, 35, 1], ['NR1037', 'STEPOUT4530', 64, 68, 0], ['NR1037', 'NR1005', 60, 75, 0], ['NR1039', 'STEPOUT4530', 62, 78, 0], ['NR1039', 'NR610', 69, 114, 0], ['NR1040', 'NR1002', 26, 22, 0], ['NR1040', 'NR1022', 24, 22, 0], ['NR1041', 'NR1000', 53, 90, 0], ['NR1041', 'NR1058', 31, 21, 0], ['NR1042', 'NR1062', 22, 35, 0], ['NR1042', 'NR1010', 37, 65, 0], ['NR1042', 'RI10A-INPIT/SME', 8, 10, 0], ['NR1042', 'RI10B-SOBRECARGA/EST', 10, 12, 0], ['NR120', 'RF12-4600-1206-1', 35, 50, 0], ['NR1042', 'RI10A-SOBRECARGA/SME', 8, 10, 0], ['NR1042', 'RI10B-INPIT/SAL', 7, 9, 0], ['NR1043', 'NR1034', 54, 115, 0], ['NR1043', 'NR1035', 33, 25, 0], ['NR1044', 'NR1057', 21, 48, 0], ['NR1044', 'NR1062', 31, 23, 0], ['NR1045', 'RAMPA CAPELLA', 211, 256, 0], ['NR1045', 'NR1003', 33, 28, 0], ['NR1046', 'NR1028', 23, 52, 0], ['NR1046', 'NR1057', 43, 33, 0], ['NR1047', 'NR1000', 9, 10, 0], ['NR1047', 'NR1050', 8, 10, 1], ['NR1048', 'NR1105', 37, 109, 0], ['NR1048', 'NR1050', 38, 47, 0], ['NR1049', 'NR633', 39, 61, 0], ['NR1049', 'NR1051', 29, 31, 0], ['NR1050', 'NR1048', 39, 89, 0], ['NR1050', 'NR1033', 14, 17, 0], ['NR1050', 'NR1047', 10, 14, 0], ['NR1051', 'NR1054', 35, 37, 0], ['NR1051', 'NR1049', 67, 111, 0], ['NR1051', 'RB1-FALLAGRIS/EST', 9, 42, 0], ['NR1051', 'RI10A-INPITGRIS/EST', 11, 14, 0], ['NR1052', 'NR1016', 44, 51, 0], ['NR1052', 'NR1053', 23, 41, 0], ['NR1053', 'NR1052', 45, 47, 0], ['NR1053', 'NR1055', 15, 32, 0], ['NR1053', 'NR1031', 19, 17, 0], ['NR1054', 'NR1051', 129, 212, 0], ['NR1054', 'NR1016', 20, 24, 0], ['NR1055', 'NR1060', 28, 60, 0], ['NR1055', 'NR1053', 12, 10, 0], ['NR1057', 'NR1046', 155, 348, 0], ['NR1057', 'NR1044', 19, 16, 0], ['NR1058', 'NR1041', 66, 122, 0], ['NR1058', 'NR1060', 24, 17, 0], ['NR1060', 'NR1058', 33, 62, 0], ['NR1060', 'NR1055', 22, 18, 1], ['NR1061', 'R CAPELLA CORTA', 49, 71, 0], ['NR1061', 'NR1030', 34, 33, 0], ['NR1062', 'NR1042', 14, 17, 0], ['NR1062', 'RI10A-INPIT/SBH', 13, 12, 0], ['NR1062', 'RI10A-INPIT/EST', 10, 8, 0], ['NR1062', 'RI10B-INPIT/SBH', 12, 11, 0], ['NR1062', 'RI10A-INPIT/SAL', 13, 26, 0], ['NR1062', 'RI10A-SOBRECARGA/SAL', 6, 8, 0], ['NR1062', 'NR1044', 92, 208, 0], ['NR1209', 'RF12-4615-1233-1', 25, 32, 0], ['NR1100', 'NR1106', 137, 218, 0], ['NR1100', 'NR1103', 39, 27, 0], ['NR1101', 'RAMPA FASE11', 28, 23, 0], ['NR1101', 'NR1104', 148, 157, 0], ['NR1102', 'RAMPA FASE11', 25, 24, 0], ['NR1102', 'NR1105', 24, 20, 0], ['NR1103', 'NR1104', 27, 19, 0], ['NR1103', 'NR1100', 147, 155, 0], ['NR1104', 'NR1103', 55, 58, 0], ['NR1104', 'NR1108', 34, 27, 0], ['NR1104', 'NR1101', 24, 18, 0], ['NR1105', 'NR1102', 143, 133, 0], ['NR1105', 'NR1048', 22, 25, 0], ['NR1106', 'NR1100', 27, 21, 0], ['NR1106', 'NR322', 39, 62, 0], ['NR1107', 'NR1033', 20, 51, 0], ['NR1107', 'NR1111', 21, 17, 1], ['NR1108', 'NR1104', 57, 76, 0], ['NR1108', 'NR1115', 30, 25, 0], ['NR1109', 'NR1112', 29, 28, 0], ['NR1109', 'NR1115', 57, 62, 0], ['NR1110', 'NR1117', 58, 96, 0], ['NR1110', 'NR1120', 33, 39, 0], ['NR1110', 'RI11-INPIT/SBL', 7, 16, 0], ['NR1110', 'RI11-SOBRECARGA/EST', 11, 14, 0], ['NR1110', 'RI11-INPIT/SME', 15, 19, 0], ['NR1110', 'RI11-INPIT/SBH', 11, 14, 0], ['NR1110', 'NR1118', 175, 217, 0], ['NR1110', 'NR1130', 43, 43, 0], ['NR1111', 'NR1107', 28, 73, 0], ['NR1111', 'NR1117', 14, 14, 0], ['NR1112', 'NR1109', 54, 94, 0], ['NR1112', 'NR1113', 26, 22, 0], ['NR1113', 'NR1112', 87, 122, 0], ['NR1113', 'NR1116', 26, 22, 0], ['NR1115', 'NR1108', 48, 53, 1], ['NR1115', 'NR1109', 24, 20, 0], ['NR1116', 'NR1113', 112, 156, 0], ['NR1116', 'NR1118', 22, 23, 0], ['NR1117', 'NR1111', 14, 32, 0], ['NR1117', 'NR1110', 43, 50, 0], ['NR1117', 'RI11-INPIT/SAL', 7, 9, 0], ['NR1117', 'RI11-INPIT/EST', 9, 12, 0], ['NR1117', 'RI11-SOBRECARGA/SBH', 11, 14, 0], ['NR1118', 'NR1116', 29, 31, 0], ['NR1118', 'NR1110', 37, 31, 0], ['NU74', 'UB1-TRANQUECORON/EST', 18, 23, 0], ['NR1120', 'NR817', 33, 40, 0], ['NR1120', 'NR1110', 38, 43, 0], ['NR1130', 'NR1110', 59, 99, 0], ['NR1130', 'RF11-4300-1101-1', 39, 38, 0], ['NR1200', 'NR294', 28, 31, 0], ['NR1200', 'NR1201', 33, 48, 0], ['NR1200', 'NR364', 39, 23, 0], ['NR1201', 'NR1200', 45, 50, 0], ['NR1201', 'NR1207', 29, 44, 0], ['NR1202', 'NR1203', 107, 116, 0], ['NR1202', 'NR1208', 54, 42, 0], ['NR1203', 'NR1212', 103, 111, 0], ['NR1203', 'NR1202', 43, 49, 0], ['NR1204', 'NR310', 51, 69, 0], ['NR1204', 'NR279', 56, 56, 0], ['NR1206', 'NR1207', 30, 39, 0], ['NR1063', 'RI10A-INPIT/SBL', 12, 15, 0], ['NR1209', 'RF12-4615-1237-1', 40, 48, 0], ['NR1207', 'NR1201', 30, 37, 0], ['NR1207', 'NR1206', 30, 31, 0], ['NR1207', 'RI12-INPIT/OXI', 21, 27, 0], ['NR1207', 'RI12-INPIT/EST', 21, 29, 0], ['NR1207', 'RI12-SOBRECARGA/EST', 23, 29, 0], ['NR1208', 'NR1202', 52, 104, 0], ['RI10A-INPIT/SBL', 'NR1063', 12, 15, 0], ['NR1056', 'RF10B-4195-1022-1', 18, 21, 0], ['NR1206', 'NR275', 22, 22, 0], ['RF10B-4195-1022-1', 'NR1056', 17, 22, 0], ['NR1059', 'NR1038', 17, 22, 0], ['NR1211', 'NR1013', 55, 47, 0], ['NR1211', 'NR1212', 37, 38, 0], ['NR1212', 'NR1211', 150, 165, 0], ['NR1212', 'NR1203', 38, 41, 0], ['NR1381', 'NR1382', 57, 75, 0], ['NR1381', 'NR1024', 58, 65, 0], ['NR1382', 'NR1027', 64, 82, 0], ['NR1382', 'NR1381', 76, 86, 0], ['NR199', 'NR214', 36, 44, 0], ['NR199', 'NR208', 50, 63, 0], ['NR199', 'NR230', 62, 82, 1], ['NR200', 'NR353', 18, 44, 0], ['NR200', 'NR314', 35, 32, 0], ['NR200', 'P2 SEMIMOVIL', 21, 20, 0], ['NR201', 'NR207', 18, 39, 0], ['NR201', 'NR271', 13, 16, 1], ['NR201', 'NR299', 47, 41, 0], ['NR202', 'NR330', 22, 15, 0], ['NR202', 'NR300', 41, 34, 0], ['NR203', 'NR339', 34, 28, 0], ['NR203', 'NR217', 49, 81, 0], ['NR204', 'NR217', 44, 53, 0], ['NR204', 'NR321', 25, 39, 0], ['NR205', 'NR333', 18, 19, 0], ['NR205', 'NR235', 26, 25, 0], ['NR205', 'NR380', 29, 35, 1], ['NR205', 'NR300', 45, 44, 0], ['NR205', 'NR381', 47, 58, 1], ['NR205', 'NR306', 48, 59, 0], ['NR206', 'NR335', 22, 16, 0], ['NR206', 'NR386', 32, 51, 0], ['NR206', 'NR371', 76, 91, 1], ['NR207', 'NR201', 21, 25, 1], ['NR207', 'NR228', 23, 44, 0], ['NR208', 'NR199', 63, 73, 1], ['NR208', 'NR331', 21, 28, 0], ['NR209', 'NR337', 27, 45, 0], ['NR209', 'RS2-FLAMENCO/SME', 77, 84, 0], ['NR210', 'RB1-GUANACOLARGO/EST', 35, 44, 1], ['NR210', 'NR214', 47, 58, 0], ['NR211', 'NR215', 31, 31, 0], ['NR211', 'RS2-4600/OXA', 18, 23, 0], ['NR211', 'NR272', 18, 25, 0], ['NR212', 'NR264', 23, 29, 1], ['NR212', 'NR227', 37, 22, 0], ['NR212', 'RS3-JUREL/MIX', 44, 51, 0], ['NR212', 'NR268', 27, 32, 1], ['NR213', 'NR339', 13, 12, 0], ['NR213', 'P3 SEMIMOVIL', 38, 47, 1], ['NR213', 'NR236', 11, 12, 0], ['NR214', 'RB1-GUANACOLARGO/EST', 99, 120, 0], ['NR214', 'NR260', 46, 60, 0], ['NR214', 'NR210', 54, 64, 1], ['NR214', 'NR199', 31, 43, 0], ['NR215', 'NR211', 26, 30, 1], ['NR215', 'NR239', 38, 58, 0], ['NR215', 'RS1-PEJERREY/MIX', 77, 66, 0], ['NR216', 'NR276', 30, 36, 0], ['NR216', 'NR268', 44, 75, 0], ['NR216', 'NR237', 15, 17, 0], ['NR216', 'C-ENTRENAMIENTO', 111, 78, 0], ['NR217', 'NR203', 29, 22, 0], ['NR217', 'NR204', 71, 99, 0], ['NR217', 'NR265', 77, 97, 0], ['NR218', 'P3 SEMIMOVIL', 35, 26, 0], ['NR218', 'NR603', 32, 39, 1], ['NR219', 'NR330', 30, 32, 1], ['NR219', 'NR329', 16, 22, 0], ['NR220', 'NR224', 24, 23, 0], ['NR220', 'NR304', 21, 23, 1], ['NR221', 'NR269', 25, 23, 0], ['NR221', 'NR355', 27, 37, 0], ['NR222', 'NR317', 74, 118, 0], ['NR222', 'NR254', 81, 89, 0], ['NR223', 'NR285', 34, 32, 0], ['NR223', 'NR224', 61, 69, 0], ['NR224', 'NR223', 32, 29, 0], ['NR224', 'NR292', 61, 59, 0], ['NR224', 'NR220', 34, 37, 1], ['NR225', 'NR327', 41, 37, 1], ['NR225', 'NR341', 148, 248, 0], ['NR225', 'NR302', 35, 39, 0], ['NR226', 'NR278', 46, 29, 0], ['NR226', 'NR363', 34, 55, 0], ['NR227', 'NR241', 77, 43, 0], ['NR227', 'NR212', 11, 14, 1], ['NR227', 'RS3-CARPA/OXA', 28, 35, 0], ['NR228', 'NR262', 15, 30, 0], ['NR228', 'NR207', 18, 22, 1], ['NR228', 'NR299', 17, 21, 1], ['NR228', 'NR274', 61, 78, 0], ['NR228', 'NR242', 13, 16, 0], ['NR229', 'NR284', 11, 16, 0], ['NR229', 'NR233', 15, 18, 0], ['NR230', 'NR199', 71, 86, 0], ['NR230', 'RB1-GUANACOMEDIO/EST', 34, 42, 1], ['NR230', 'RB1-HUINQUINEUTR/EST', 12, 15, 1], ['NR230', 'RB1-HUINQUICORTO/EST', 7, 10, 0], ['NR231', 'NR246', 89, 60, 0], ['NR231', 'RS1-4600/OXB', 30, 38, 0], ['NR231', 'RS4-4600/OXB', 43, 31, 0], ['NR231', 'RS1-4600/MIX', 20, 25, 0], ['NR231', 'RS2-4600/OXB', 33, 41, 0], ['NR231', 'RS1-4600/OXA', 44, 53, 0], ['NR231', 'NR267', 45, 50, 0], ['NR231', 'NR1001', 34, 38, 0], ['NR231', 'ESTAC 4600', 44, 85, 0], ['NR232', 'NR356', 77, 100, 0], ['NR232', 'NR269', 21, 24, 0], ['NR233', 'NR229', 11, 15, 0], ['NR233', 'RB1-CHINCHILLA/EST', 20, 25, 0], ['NR233', 'RS1-CHINCHILLA/SME', 69, 88, 0], ['NR233', 'NR360', 37, 47, 0], ['NR234', 'NR333', 46, 50, 0], ['NR234', 'NR297', 74, 72, 0], ['NR235', 'NR236', 13, 12, 0], ['NR235', 'NR205', 21, 22, 0], ['NR236', 'NR235', 12, 12, 0], ['NR236', 'NR247', 76, 56, 0], ['NR236', 'NR213', 15, 14, 0], ['NR237', 'NR216', 10, 13, 1], ['NR237', 'NR272', 20, 20, 0], ['NR238', 'NR303', 23, 30, 0], ['NR238', 'NR242', 38, 48, 1], ['NR238', 'NR271', 27, 25, 0], ['NR239', 'NR264', 93, 57, 0], ['NR239', 'NR215', 26, 33, 1], ['NR239', 'RS3-TOYO/LIX', 54, 55, 0], ['NR239', 'RS3-SALMON/OXA', 64, 88, 0], ['NR240', 'NR331', 12, 15, 1], ['NR240', 'RB1-GUANACOMEDIO/EST', 7, 10, 0], ['NR241', 'NR227', 24, 29, 1], ['NR241', 'NR286', 107, 64, 0], ['NR242', 'NR238', 48, 64, 0], ['NR242', 'NR248', 16, 20, 1], ['NR242', 'NR228', 18, 18, 0], ['NR243', 'NR246', 46, 50, 0], ['NR243', 'RB1-PUMANEUTRO/EST', 23, 31, 0], ['NR243', 'RB3-NEUTRO/EST', 16, 20, 0], ['NR244', 'NR337', 12, 14, 0], ['NR244', 'NR248', 9, 14, 0], ['NR245', 'NR375', 41, 78, 0], ['NR245', 'NR391', 18, 33, 0], ['NR245', 'RB1-ZORRO4675/EST', 68, 137, 0], ['NR246', 'NR231', 59, 52, 0], ['NR246', 'RS5-4600/OXB', 22, 26, 0], ['NR246', 'RS3-4600/OXB', 30, 32, 0], ['NR246', 'RS2-4600/MIX', 33, 37, 0], ['NR246', 'NR287', 25, 21, 0], ['NR246', 'NR243', 34, 35, 1], ['NR246', 'NR273', 27, 33, 1], ['NR246', 'PLAT PUMA 2', 51, 44, 0], ['NR247', 'NR236', 46, 58, 0], ['NR247', 'NR261', 49, 36, 0], ['NR248', 'NR244', 10, 11, 0], ['NR248', 'NR242', 33, 25, 0], ['NR248', 'NR280', 8, 14, 0], ['NR248', 'NR281', 24, 20, 1], ['NR249', 'NR322', 8, 6, 0], ['NR249', 'PROY. CRUZ', 4, 5, 0], ['NR249', 'NR282', 24, 34, 0], ['NR249', 'NR293', 11, 13, 0], ['NR250', 'NR309', 12, 10, 0], ['NR250', 'P4 SEMIMOVIL', 24, 33, 1], ['NR250', 'NR280', 16, 17, 1], ['NR251', 'NR273', 24, 14, 0], ['NR251', 'NR286', 51, 64, 0], ['NR252', 'NR288', 35, 45, 0], ['NR252', 'NR274', 39, 46, 0], ['NR252', 'NR314', 60, 69, 0], ['NR253', 'NR276', 28, 29, 0], ['NR253', 'NR295', 38, 22, 0], ['NR253', 'NR287', 31, 39, 1], ['NR254', 'NR222', 81, 70, 0], ['NR254', 'NR342', 40, 46, 0], ['NR256', 'NR257', 25, 30, 0], ['NR256', 'NR320', 30, 37, 1], ['NR256', 'NR325', 41, 44, 0], ['NR256', 'RS1-PARINA/SME', 51, 107, 0], ['NR256', 'RS2-4550/SBL', 14, 18, 0], ['NR257', 'NR256', 20, 25, 1], ['NR257', 'NR285', 66, 84, 0], ['NR257', 'RS1-TAGUA/SAL', 30, 50, 0], ['NR257', 'NR320', 12, 15, 1], ['NR258', 'P4 SEMIMOVIL', 14, 17, 0], ['NR258', 'NR262', 8, 10, 1], ['NR259', 'NR277', 21, 24, 0], ['NR259', 'NR325', 43, 49, 0], ['NR260', 'NR214', 34, 31, 0], ['NR260', 'NR356', 35, 42, 0], ['NR261', 'NR247', 30, 39, 0], ['NR261', 'NR603', 22, 15, 0], ['NR261', 'NR604', 9, 10, 0], ['NR262', 'NR228', 18, 21, 1], ['NR262', 'NR258', 12, 9, 0], ['NR262', 'NR280', 8, 18, 0], ['NR263', 'NR278', 31, 42, 1], ['NR263', 'NR282', 23, 22, 0], ['NR264', 'NR239', 38, 47, 1], ['NR264', 'NR212', 69, 41, 0], ['NR265', 'NR217', 66, 81, 0], ['NR265', 'NR388', 70, 103, 0], ['NR265', 'NR387', 66, 84, 1], ['NR266', 'NR296', 32, 28, 0], ['NR266', 'NR290', 40, 56, 1], ['NR267', 'PLAT PUMA 1', 34, 40, 0], ['NR267', 'NR231', 26, 42, 0], ['NR267', 'NR346', 19, 24, 0], ['NR267', 'NR378', 14, 18, 1], ['NR268', 'NR216', 36, 44, 1], ['NR268', 'NR212', 19, 35, 0], ['NR269', 'NR221', 24, 26, 0], ['NR269', 'NR232', 32, 29, 0], ['NR270', 'NR308', 26, 33, 0], ['NR270', 'NR354', 37, 46, 0], ['NR270', 'NR361', 45, 54, 1], ['NR271', 'NR201', 17, 23, 0], ['NR271', 'NR307', 9, 12, 1], ['NR271', 'PLAT LLAMA 2', 33, 65, 0], ['NR271', 'NR238', 19, 23, 1], ['NR272', 'NR211', 37, 34, 0], ['NR272', 'NR237', 13, 16, 1], ['NR273', 'NR251', 10, 13, 1], ['NR273', 'NR246', 30, 37, 0], ['NR273', 'NR287', 9, 12, 1], ['NR273', 'NR374', 46, 40, 0], ['NR274', 'NR252', 35, 44, 0], ['NR274', 'NR228', 64, 78, 0], ['NR274', 'NR357', 112, 198, 0], ['NR276', 'NR253', 19, 24, 0], ['NR276', 'NR216', 37, 38, 0], ['NR277', 'NR259', 19, 24, 0], ['NR277', 'NR284', 20, 23, 0], ['NR278', 'NR263', 28, 22, 0], ['NR278', 'NR288', 5, 5, 0], ['NR278', 'NR282', 33, 31, 1], ['NR278', 'NR323', 18, 23, 1], ['NR278', 'NR599', 50, 52, 0], ['NR278', 'NR226', 40, 81, 0], ['NR279', 'NR328', 105, 103, 0], ['NR279', 'NR1204', 41, 56, 0], ['NR280', 'NR248', 12, 19, 0], ['NR280', 'NR262', 9, 11, 1], ['NR280', 'NR250', 12, 11, 0], ['NR281', 'NR322', 15, 12, 1], ['NR281', 'NR248', 72, 93, 0], ['NR282', 'NR263', 20, 22, 1], ['NR282', 'NR249', 18, 14, 0], ['NR282', 'NR278', 53, 58, 0], ['NR283', 'NR332', 46, 52, 0], ['NR283', 'NR366', 39, 38, 0], ['NR283', 'RS1-PACHICA/OXI', 42, 53, 0], ['NR284', 'NR277', 18, 23, 0], ['NR284', 'NR229', 16, 18, 0], ['NR284', 'PLAT COBRE', 16, 18, 0], ['NR284', 'RS1-GAVIOTA/SAL', 29, 33, 0], ['NR285', 'NR307', 38, 53, 0], ['NR285', 'NR289', 11, 13, 1], ['NR285', 'NR223', 53, 50, 0], ['NR285', 'NR257', 73, 90, 1], ['NR286', 'NR251', 150, 88, 0], ['NR286', 'NR241', 48, 57, 0], ['NR287', 'NR246', 14, 17, 1], ['NR287', 'NR253', 52, 53, 0], ['NR287', 'NR273', 15, 14, 0], ['NR288', 'NR278', 4, 6, 0], ['NR288', 'NR252', 29, 32, 1], ['NR289', 'NR303', 37, 47, 1], ['NR289', 'NR285', 9, 12, 0], ['NR289', 'NR320', 61, 78, 0], ['NR290', 'NR266', 18, 15, 0], ['NR290', 'NR298', 15, 15, 1], ['NR290', 'RB1-RAMPAZORRO/EST', 43, 48, 0], ['NR291', 'NR321', 23, 30, 0], ['NR291', 'RS1-SURI/SBH', 14, 23, 0], ['NR292', 'NR224', 38, 40, 1], ['NR292', 'NR298', 501, 620, 1], ['NR293', 'NR249', 16, 18, 1], ['NR293', 'NR322', 18, 24, 0], ['NR293', 'NR607', 33, 32, 0], ['NR294', 'NR301', 102, 76, 0], ['NR294', 'NR319', 16, 12, 0], ['NR294', 'NR1200', 56, 60, 0], ['NR295', 'RS3-4600/MIX', 49, 27, 0], ['NR295', 'NR253', 18, 23, 0], ['NR296', 'NR304', 50, 38, 1], ['NR296', 'NR266', 53, 59, 1], ['NR297', 'NR1009', 32, 36, 0], ['NR297', 'NR234', 52, 59, 0], ['NR298', 'NR292', 80, 63, 1], ['NR298', 'NR290', 22, 24, 0], ['NR299', 'PLAT LLAMA 1', 35, 29, 0], ['NR299', 'NR228', 34, 51, 0], ['NR299', 'NR201', 25, 32, 1], ['NR300', 'NR202', 32, 28, 0], ['NR300', 'NR333', 45, 40, 0], ['NR300', 'NR205', 46, 53, 1], ['NR301', 'NR310', 123, 92, 0], ['NR301', 'NR294', 63, 62, 0], ['NR302', 'NR333', 27, 32, 0], ['NR302', 'NR327', 7, 9, 1], ['NR302', 'NR380', 12, 16, 0], ['NR302', 'NR225', 66, 75, 1], ['NR303', 'NR238', 27, 33, 1], ['NR303', 'NR307', 9, 13, 0], ['NR303', 'NR289', 35, 46, 0], ['NR304', 'NR220', 16, 14, 0], ['NR304', 'NR296', 409, 577, 1], ['NR305', 'NR336', 22, 19, 0], ['NR305', 'NR326', 24, 24, 0], ['NR306', 'NR205', 42, 53, 1], ['NR306', 'NR345', 29, 40, 0], ['NR307', 'NR303', 9, 10, 0], ['NR307', 'NR271', 11, 16, 0], ['NR307', 'NR285', 46, 58, 1], ['NR308', 'NR270', 21, 27, 0], ['NR308', 'NR603', 30, 39, 0], ['NR309', 'PROY. CRUZ', 14, 13, 0], ['NR309', 'NR250', 36, 51, 1], ['NR310', 'NR301', 59, 58, 0], ['NR310', 'NR1204', 72, 62, 0], ['NR311', 'NR599', 8, 10, 1], ['NR311', 'RS1-STOCKCR04/SME', 12, 31, 0], ['NR311', 'NR353', 7, 10, 0], ['NR311', 'RS2-STOCKCR04/SME', 21, 44, 0], ['NR312', 'NU28', 32, 43, 0], ['NR312', 'NR324', 41, 42, 0], ['NR312', 'NR1011', 126, 144, 0], ['NR313', 'NR315', 54, 78, 0], ['NR313', 'NR319', 31, 32, 0], ['NR313', 'NR366', 42, 41, 0], ['NR314', 'PLAT INVIERNO', 30, 42, 0], ['NR314', 'NR252', 45, 54, 0], ['NR314', 'TALLER BAYLAC', 256, 94, 0], ['NR314', 'NR348', 35, 29, 0], ['NR314', 'WASH SHOP', 125, 86, 0], ['NR314', 'NR200', 44, 57, 0], ['NR315', 'NR317', 53, 85, 1], ['NR315', 'NR313', 65, 68, 0], ['NR316', 'NR334', 31, 45, 0], ['NR316', 'NR337', 61, 75, 0], ['NR317', 'NR315', 84, 128, 1], ['NR317', 'NR222', 52, 71, 0], ['NR318', 'NR382', 43, 49, 0], ['NR318', 'RB1-PUMA/EST', 34, 44, 0], ['NR319', 'NR294', 36, 46, 0], ['NR319', 'TALLER BAYLAC', 36, 41, 1], ['NR319', 'NR313', 28, 32, 0], ['NR320', 'NR289', 60, 75, 1], ['NR320', 'NR256', 30, 34, 0], ['NR320', 'NR257', 16, 25, 0], ['NR321', 'NR204', 19, 24, 0], ['NR321', 'NR291', 22, 36, 0], ['NR321', 'NR349', 31, 127, 0], ['NR322', 'NR281', 43, 39, 0], ['NR322', 'NR249', 13, 26, 0], ['NR322', 'NR1106', 11, 8, 0], ['NR322', 'NR293', 16, 21, 1], ['NR323', 'NR278', 11, 18, 0], ['NR323', 'NR347', 21, 22, 1], ['NR323', 'NR340', 20, 22, 1], ['NR324', 'NR312', 100, 46, 0], ['NR324', 'NR328', 90, 80, 0], ['NR325', 'NR256', 29, 35, 0], ['NR325', 'RS4-CONDOR/SME', 29, 27, 0], ['NR325', 'RS1-AGUILA/SAA', 26, 32, 0], ['NR325', 'RS2-GOLONDRINA/SME', 58, 91, 0], ['NR325', 'NR259', 36, 46, 0], ['NR326', 'NR305', 34, 27, 0], ['NR326', 'NR339', 19, 21, 0], ['NR327', 'NR225', 66, 85, 0], ['NR327', 'NR333', 22, 27, 1], ['NR327', 'NR302', 10, 10, 0], ['NR328', 'NR279', 60, 55, 0], ['NR328', 'NR324', 254, 112, 0], ['NR328', 'RB1-PATIOPALA/EST', 27, 29, 0], ['NR329', 'NR219', 21, 25, 1], ['NR329', 'NR914', 19, 28, 0], ['NR330', 'NR202', 113, 115, 0], ['NR330', 'NR219', 21, 21, 0], ['NR330', 'NR907', 29, 32, 1], ['NR331', 'NR208', 30, 35, 1], ['NR331', 'NR240', 11, 14, 0], ['NR332', 'RS1-LIRIMA/MIX', 51, 70, 0], ['NR332', 'NR338', 50, 53, 0], ['NR332', 'NR283', 30, 29, 0], ['NR333', 'NR205', 16, 18, 0], ['NR333', 'NR302', 30, 36, 1], ['NR333', 'NR327', 26, 31, 0], ['NR333', 'NR234', 58, 60, 0], ['NR333', 'NR300', 22, 20, 1], ['NR334', 'NR316', 39, 35, 0], ['NR334', 'NR336', 29, 40, 0], ['NR335', 'NR343', 36, 35, 0], ['NR335', 'NR206', 37, 55, 0], ['NR336', 'NR334', 28, 24, 0], ['NR336', 'NR305', 22, 21, 0], ['NR336', 'NR350', 64, 55, 0], ['NR336', 'NR351', 35, 42, 0], ['NR337', 'NR316', 78, 94, 0], ['NR337', 'NR244', 9, 16, 0], ['NR337', 'NR209', 31, 37, 0], ['NR337', 'RB1-FLAMENCO/EST', 49, 78, 0], ['NR338', 'NR332', 30, 32, 0], ['NR338', 'NR352', 34, 59, 0], ['NR339', 'NR326', 22, 22, 0], ['NR339', 'NR203', 51, 72, 0], ['NR339', 'NR213', 9, 10, 0], ['NR340', 'NR599', 5, 13, 0], ['NR340', 'NR347', 10, 6, 0], ['NR340', 'NR323', 13, 14, 0], ['NR340', 'NR353', 11, 13, 1], ['NR341', 'NR225', 37, 39, 0], ['NR341', 'NR343', 72, 89, 0], ['NR342', 'NR357', 42, 67, 0], ['NR342', 'NR254', 42, 34, 0], ['NR343', 'NR382', 30, 39, 0], ['NR343', 'NR335', 41, 53, 0], ['NR343', 'NR341', 42, 39, 0], ['NR344', 'NR381', 29, 32, 0], ['NR344', 'RB2-ALPACACORTO/EST', 51, 64, 1], ['NR345', 'NR306', 32, 40, 1], ['NR345', 'RB2-ALPACACORTO/EST', 54, 62, 0], ['NR346', 'NR267', 19, 24, 1], ['NR346', 'RB1-PUMA4600/EST', 13, 15, 0], ['NR347', 'CS03', 13, 10, 0], ['NR347', 'NR323', 16, 32, 0], ['NR347', 'NR340', 7, 15, 0], ['NR348', 'NR314', 26, 33, 0], ['NR348', 'TALLER ROSARIO', 13, 12, 0], ['NR349', 'NR321', 35, 40, 0], ['NR349', 'RS1-SURI/SBH', 11, 14, 0], ['NR350', 'NR336', 30, 37, 0], ['NR350', 'RB1-ALPACA2/EST', 13, 21, 0], ['NR351', 'RB1-ALPACA2/EST', 10, 13, 1], ['NR351', 'NR336', 28, 56, 0], ['NR352', 'NR338', 23, 20, 0], ['NR352', 'NR379', 52, 90, 0], ['NR352', 'RB1-RAMPALIRIMA/EST', 8, 13, 0], ['NR353', 'NR311', 5, 6, 1], ['NR353', 'CS03', 6, 12, 0], ['NR353', 'NR340', 12, 18, 0], ['NR353', 'NR200', 21, 29, 0], ['NR354', 'NR270', 33, 41, 1], ['NR354', 'NR365', 16, 21, 0], ['NR355', 'NR221', 35, 71, 0], ['NR355', 'NR370', 18, 24, 0], ['NR356', 'NR232', 43, 55, 0], ['NR356', 'NR367', 96, 111, 0], ['NR356', 'NR260', 27, 34, 0], ['NR357', 'NR274', 55, 56, 0], ['NR357', 'NR342', 45, 33, 0], ['NR358', 'NR370', 32, 67, 0], ['NR358', 'NR362', 26, 34, 0], ['NR360', 'RB2-ALPACA/EST', 39, 50, 1], ['NR360', 'NR361', 32, 39, 0], ['NR360', 'NR233', 35, 45, 0], ['NR361', 'NR360', 25, 31, 1], ['NR361', 'NR270', 38, 48, 0], ['NR362', 'NR358', 28, 57, 0], ['NR363', 'NR226', 49, 36, 0], ['NR363', 'NR364', 39, 80, 0], ['NR363', 'RB1-CAMINOCHANCA/EST', 14, 16, 0], ['NR364', 'NR363', 46, 29, 0], ['NR364', 'NR1200', 130, 280, 0], ['NR1200', 'NR1205', 64, 62, 0], ['NR365', 'NR354', 15, 19, 1], ['NR365', 'RB2-ALPACA/EST', 16, 21, 0], ['NR366', 'NR313', 22, 21, 0], ['NR366', 'NR283', 68, 69, 0], ['NR367', 'NR369', 43, 67, 0], ['NR367', 'NR356', 54, 67, 0], ['NR367', 'NR380', 49, 55, 0], ['NR369', 'NR367', 48, 48, 0], ['NR369', 'NR373', 30, 38, 0], ['NR370', 'NR355', 27, 52, 0], ['NR370', 'NR358', 27, 37, 0], ['NR1213', 'NR1214', 34, 40, 0], ['NR371', 'NR384', 66, 68, 0], ['NR371', 'NR206', 53, 75, 0], ['NR373', 'NR369', 37, 43, 0], ['NR373', 'NR383', 64, 84, 0], ['NR374', 'NR273', 34, 27, 0], ['NR374', 'NR384', 90, 81, 0], ['NR375', 'NR379', 30, 38, 0], ['NR375', 'NR245', 62, 92, 0], ['NR378', 'RB1-PUMA4600/EST', 14, 16, 1], ['NR378', 'NR267', 14, 17, 0], ['NR379', 'NR352', 34, 34, 0], ['NR379', 'NR375', 62, 102, 0], ['NR380', 'NR367', 53, 55, 0], ['NR380', 'NR205', 26, 28, 0], ['NR380', 'NR302', 13, 16, 1], ['NR381', 'NR205', 51, 59, 0], ['NR381', 'NR344', 21, 26, 1], ['NR382', 'NR318', 38, 50, 0], ['NR382', 'NR343', 34, 38, 0], ['NR383', 'NR373', 62, 77, 0], ['NR383', 'RB1-GUANACOSUP/EST', 51, 65, 0], ['NR384', 'NR374', 66, 70, 0], ['NR384', 'NR385', 65, 73, 1], ['NR384', 'NR371', 59, 73, 0], ['NR384', 'RB2-PUMA2/EST', 96, 115, 0], ['NR385', 'NR386', 57, 74, 0], ['NR385', 'NR384', 51, 69, 0], ['NR386', 'NR385', 54, 87, 0], ['NR386', 'NR206', 33, 56, 0], ['NR387', 'NR265', 62, 142, 0], ['NR387', 'RS1-4550/SBL', 92, 105, 1], ['NR388', 'NR265', 61, 77, 1], ['NR388', 'RS1-4550/SBL', 112, 154, 0], ['NR389', 'NR391', 20, 32, 0], ['NR389', 'RB1-ZORRO/EST', 47, 52, 1], ['NR390', 'NR391', 21, 21, 1], ['NR390', 'RB1-ZORRO/EST', 51, 81, 0], ['NR391', 'NR245', 24, 33, 0], ['NR391', 'NR389', 20, 26, 1], ['NR391', 'NR390', 30, 43, 0], ['NR599', 'NR278', 21, 17, 1], ['NR599', 'NR311', 11, 26, 0], ['NR599', 'NR340', 8, 5, 0], ['NR600', 'NR601', 25, 24, 0], ['NR600', 'NR636', 180, 196, 0], ['NR601', 'NR600', 76, 84, 0], ['NR601', 'NR637', 19, 27, 0], ['NR603', 'NR308', 27, 35, 0], ['NR603', 'NR261', 10, 13, 0], ['NR603', 'NR604', 5, 7, 1], ['NR603', 'NR218', 49, 37, 0], ['NR604', 'NR261', 6, 9, 1], ['NR604', 'NR603', 5, 7, 0], ['NR605', 'NR650', 80, 113, 0], ['NR605', 'RI1-INPITFASE6/SME', 11, 16, 0], ['NR605', 'RI8B2-INPIT/SME', 5, 4, 0], ['NR605', 'RI8B2-INPIT/SAL', 7, 7, 0], ['NR606', 'NR617', 27, 42, 0], ['NR606', 'NR843', 13, 16, 0], ['NR607', 'NR608', 24, 26, 0], ['NR607', 'NR293', 28, 35, 0], ['NR608', 'NR610', 17, 19, 0], ['NR608', 'NR607', 28, 37, 0], ['NR610', 'NR645', 26, 19, 0], ['NR610', 'NR608', 22, 29, 0], ['NR610', 'NR1039', 92, 106, 1], ['NR617', 'NR618', 110, 113, 0], ['NR617', 'NR606', 17, 19, 0], ['NR618', 'NR617', 26, 32, 0], ['NR618', 'NR620', 124, 231, 0], ['NR620', 'NR618', 38, 35, 0], ['NR620', 'NR622', 23, 36, 0], ['NR622', 'NR919', 174, 163, 0], ['NR622', 'NR620', 24, 39, 0], ['NR633', 'NR641', 44, 63, 0], ['NR633', 'NR634', 25, 24, 0], ['NR633', 'NR1049', 33, 37, 0], ['NR634', 'NR635', 48, 30, 0], ['NR634', 'NR633', 21, 24, 0], ['NR635', 'NR634', 177, 247, 0], ['NR635', 'NR636', 28, 27, 0], ['NR636', 'NR635', 125, 144, 0], ['NR636', 'NR600', 32, 31, 0], ['NR637', 'NR601', 56, 50, 0], ['NR637', 'RAMPA TUNEL', 26, 38, 0], ['NR640', 'RAMPA TUNEL', 94, 90, 0], ['NR640', 'NR919', 20, 26, 0], ['NR641', 'RAMPA NORTE', 198, 333, 0], ['NR641', 'NR633', 31, 39, 0], ['NR643', 'NR644', 83, 114, 0], ['NR643', 'RAMPA NORTE', 44, 27, 0], ['NR644', 'NR643', 35, 23, 0], ['NR644', 'NR645', 71, 64, 0], ['NR645', 'NR610', 55, 80, 0], ['NR645', 'NR644', 36, 26, 0], ['NR650', 'NR605', 15, 12, 0], ['NR650', 'NR913', 19, 24, 1], ['NR700', 'NR701', 95, 131, 1], ['NR700', 'NR710', 117, 138, 0], ['NR701', 'NR700', 121, 140, 1], ['NR701', 'NR709', 81, 94, 0], ['NR702', 'NR703', 48, 42, 0], ['NR702', 'NR708', 55, 70, 0], ['NR703', 'NR702', 83, 94, 0], ['NR703', 'NR706', 78, 84, 0], ['NR704', 'NR707', 53, 61, 0], ['NR705', 'NR706', 58, 82, 0], ['NR705', 'NR707', 18, 24, 0], ['NR706', 'NR705', 27, 22, 0], ['NR706', 'SI1-ROSARIOSUR/EST', 17, 24, 0], ['NR706', 'NR703', 101, 112, 0], ['NR706', 'SB1-NANDU/EST', 25, 29, 0], ['NR707', 'NR704', 42, 54, 0], ['NR707', 'NR705', 23, 27, 0], ['NR708', 'NR702', 57, 72, 0], ['NR708', 'NR709', 54, 52, 0], ['NR709', 'NR708', 76, 82, 0], ['NR709', 'NR701', 62, 88, 0], ['NR710', 'NR711', 47, 64, 0], ['NR710', 'NR700', 91, 113, 0], ['NR711', 'NR710', 60, 69, 0], ['NR711', 'NR712', 38, 54, 0], ['NR712', 'NR711', 49, 57, 0], ['NR712', 'NR713', 52, 64, 0], ['NR713', 'NR712', 47, 59, 0], ['NR713', 'NU28', 63, 60, 1], ['NR801', 'NR802', 52, 64, 0], ['NR801', 'NR803', 26, 16, 0], ['NR802', 'NR808', 63, 76, 0], ['NR802', 'NR801', 31, 18, 0], ['NR803', 'NR801', 45, 60, 0], ['NR803', 'NR835', 23, 14, 0], ['NR808', 'NR802', 34, 21, 0], ['NR808', 'NR824', 62, 76, 1], ['NR817', 'NR824', 27, 16, 0], ['NR817', 'R CAPELLA CORTA', 25, 27, 0], ['NR817', 'NR1120', 39, 43, 1], ['NR823', 'NR825', 119, 238, 0], ['NR823', 'NR826', 43, 18, 0], ['NR824', 'NR808', 37, 22, 0], ['NR824', 'NR817', 42, 45, 0], ['NR825', 'NR845', 109, 149, 0], ['NR825', 'NR823', 42, 17, 0], ['NR826', 'NR823', 37, 72, 0], ['NR826', 'NR910', 25, 16, 0], ['NR835', 'NR839', 25, 14, 0], ['NR835', 'NR803', 36, 39, 0], ['NR839', 'NR835', 44, 50, 0], ['NR839', 'NR902', 29, 16, 0], ['NR843', 'NR606', 19, 24, 0], ['NR843', 'NR844', 26, 27, 0], ['NR843', 'NR846', 35, 28, 0], ['NR844', 'NR843', 109, 182, 0], ['NR844', 'NR845', 19, 17, 1], ['NR845', 'NR844', 47, 63, 0], ['NR845', 'NR825', 25, 21, 0], ['NR846', 'NR843', 69, 105, 0], ['NR846', 'NR908', 56, 63, 1], ['NR901', 'NR906', 33, 42, 0], ['NR901', 'NR910', 14, 15, 0], ['NR902', 'NR839', 48, 50, 1], ['NR902', 'NR903', 39, 26, 0], ['NR903', 'NR902', 71, 67, 0], ['NR1214', 'NR1209', 20, 26, 0], ['NR903', 'NR909', 32, 21, 0], ['NR904', 'NR909', 48, 64, 0], ['NR904', 'NR908', 16, 11, 0], ['NR905', 'PLAT CAPELLA', 14, 17, 0], ['NR905', 'NR907', 23, 33, 0], ['NR905', 'NR923', 15, 18, 1], ['NR906', 'NR908', 45, 59, 0], ['NR906', 'NR901', 17, 11, 0], ['NR907', 'NR330', 37, 49, 0], ['NR907', 'NR905', 28, 35, 1], ['NR908', 'NR904', 22, 22, 0], ['NR908', 'NR846', 72, 80, 1], ['NR908', 'NR906', 21, 14, 0], ['NR909', 'NR904', 27, 16, 0], ['NR909', 'NR903', 52, 54, 0], ['NR910', 'NR901', 20, 21, 0], ['NR910', 'NR912', 25, 56, 0], ['NR910', 'NR826', 12, 20, 0], ['NR912', 'NR910', 28, 44, 0], ['NR912', 'RI9B-INPIT/SME', 7, 9, 0], ['NR912', 'RI9B-INPIT/SAL', 5, 7, 0], ['NR912', 'RI9B-SOBRECARGA/SAL', 5, 9, 0], ['NR912', 'NR913', 32, 27, 0], ['NR913', 'NR912', 50, 83, 0], ['NR913', 'NR650', 25, 29, 1], ['NR913', 'NR921', 25, 28, 0], ['NR914', 'NR329', 26, 30, 1], ['NR914', 'NR915', 28, 33, 0], ['NR915', 'NR914', 22, 22, 1], ['NR915', 'NR1034', 25, 29, 0], ['NR919', 'NR640', 198, 186, 0], ['NR919', 'NR622', 18, 25, 0], ['NR921', 'NR913', 17, 35, 0], ['NR921', 'RF9B-4030-904-1', 54, 60, 0], ['NR923', 'NR905', 11, 16, 0], ['NR923', 'NR1034', 28, 30, 1], ['NR1208', 'NR1213', 49, 62, 0], ['NR1213', 'NR1208', 53, 72, 0], ['NR1205', 'NR120', 34, 48, 0], ['NR120', 'NR1205', 74, 54, 0], ['NR1209', 'NR1214', 29, 41, 0], ['NU01', 'NU26', 110, 158, 0], ['NU01', 'NU02', 52, 57, 0], ['NU02', 'NU01', 69, 106, 0], ['NU02', 'NU25', 59, 58, 0], ['NU03', 'NU15', 43, 40, 0], ['NU03', 'NU05', 252, 356, 0], ['NU04', 'NU29', 26, 32, 0], ['NU04', 'US1-JOTE/SAL', 43, 54, 0], ['NU04', 'NU30', 32, 35, 0], ['NU04', 'NU60', 105, 98, 0], ['NU05', 'NU06', 52, 59, 0], ['NU05', 'NU03', 45, 36, 0], ['NU06', 'NU05', 41, 50, 0], ['NU06', 'NU07', 219, 308, 0], ['NU07', 'NU09', 34, 39, 0], ['NU07', 'NU06', 36, 28, 0], ['NU08', 'NU46', 63, 51, 0], ['NU08', 'NU14', 35, 42, 0], ['NU08', 'NU16', 93, 105, 0], ['NU09', 'NU10', 29, 33, 0], ['NU09', 'NU07', 20, 18, 0], ['NU09', 'US1-4210/SBL', 62, 72, 1], ['NU09', 'US1-4210/MIX', 50, 51, 0], ['NU10', 'NU09', 22, 22, 0], ['NU10', 'NU11', 74, 101, 0], ['NU11', 'NU10', 42, 35, 0], ['NU11', 'NU12', 82, 97, 0], ['NU12', 'NU11', 47, 41, 0], ['NU12', 'NU13', 98, 108, 0], ['NU13', 'NU12', 63, 58, 0], ['NU13', 'NU14', 110, 140, 0], ['NU14', 'NU13', 63, 54, 0], ['NU14', 'NU18', 73, 75, 0], ['NU14', 'NU41', 44, 34, 0], ['NU14', 'NU44', 14, 11, 0], ['NU14', 'NU08', 25, 25, 0], ['NU15', 'NU03', 65, 69, 0], ['NU15', 'NU45', 45, 36, 0], ['NU150', 'NU25', 76, 115, 0], ['NU150', 'NU24', 40, 35, 0], ['NU16', 'TALLER UJINA', 58, 66, 0], ['NU16', 'NU08', 68, 94, 0], ['NU17', 'NU46', 73, 81, 0], ['NU17', 'NU47', 77, 68, 0], ['NU17', 'UB1-PEBBLE/EST', 35, 39, 0], ['NU18', 'NU19', 106, 185, 0], ['NU18', 'NU14', 53, 35, 0], ['NU18', 'PLAT INVIERNO UJINA', 20, 23, 0], ['NU19', 'NU20', 84, 123, 0], ['NU19', 'NU18', 94, 102, 0], ['NU20', 'NU19', 47, 36, 0], ['NU20', 'NU38', 96, 145, 0], ['NU21', 'NU37', 24, 40, 0], ['NU21', 'NU38', 57, 57, 0], ['NU22', 'NU35', 41, 48, 1], ['NU22', 'NU37', 62, 76, 0], ['NU22', 'NU36', 39, 70, 0], ['NU23', 'NU24', 126, 191, 0], ['NU23', 'NU36', 47, 45, 1], ['NU24', 'NU23', 94, 90, 1], ['NU24', 'NU34', 92, 67, 0], ['NU24', 'NU150', 66, 93, 0], ['NU25', 'NU02', 84, 123, 0], ['NU25', 'NU150', 55, 58, 0], ['NU26', 'NU01', 69, 60, 0], ['NU26', 'NU57', 54, 83, 0], ['NU27', 'NU28', 56, 72, 0], ['NU27', 'NU57', 52, 51, 1], ['NU28', 'NR312', 43, 61, 0], ['NU28', 'NU27', 42, 42, 1], ['NU28', 'NR713', 92, 99, 1], ['NU28', 'NU56', 46, 45, 0], ['NU29', 'PLAT BASE UJINA', 33, 28, 0], ['NU29', 'NU39', 53, 60, 0], ['NU29', 'US1-4335/MIX', 70, 67, 1], ['NU29', 'NU04', 38, 35, 0], ['NU29', 'US1-4330/OXA', 41, 51, 0], ['NU29', 'US1-4330/OXB', 49, 61, 0], ['NU29', 'US1-4335/OXB', 71, 76, 0], ['NU29', 'PLAT CANCHA10', 48, 89, 0], ['NU30', 'NU04', 24, 42, 0], ['NU30', 'NU42', 24, 26, 0], ['NU30', 'US1-TAMARUGO/SBA', 138, 173, 0], ['NU30', 'US2-4335/MIX', 112, 146, 0], ['NU30', 'US2-4335/SBH', 123, 152, 0], ['NU30', 'US1-LLARETA/SBH', 33, 40, 0], ['NU30', 'US1-TAMARUGO/SBH', 47, 59, 0], ['NU30', 'NU60', 89, 98, 0], ['NU30', 'UI1-INPITUJINA/SBH', 31, 40, 0], ['NU31', 'NU32', 93, 85, 0], ['NU31', 'NU42', 33, 44, 0], ['NU32', 'NU31', 75, 97, 0], ['NU32', 'NU33', 116, 99, 0], ['NU33', 'NU32', 93, 109, 0], ['NU33', 'P1 UJI', 42, 45, 0], ['NU33', 'US1-CHACHACOMA/SME', 86, 92, 0], ['NU34', 'NU24', 432, 610, 1], ['NU34', 'NU35', 46, 55, 0], ['NU35', 'NU34', 60, 69, 1], ['NU35', 'NU22', 31, 33, 0], ['NU36', 'NU23', 72, 96, 0], ['NU36', 'NU22', 21, 17, 1], ['NU37', 'NU22', 89, 128, 0], ['NU37', 'NU21', 30, 29, 0], ['NU38', 'NU21', 45, 76, 0], ['NU38', 'NU20', 73, 86, 0], ['NU38', 'US1-4460/MIX', 21, 24, 0], ['NU39', 'NU29', 66, 65, 0], ['NU39', 'NU44', 23, 26, 0], ['NU40', 'NU44', 35, 27, 0], ['NU40', 'NU71', 53, 57, 1], ['NU41', 'NU14', 31, 62, 0], ['NU41', 'CS01', 12, 10, 0], ['NU41', 'NU58', 24, 62, 0], ['NU42', 'NU43', 50, 59, 0], ['NU42', 'NU31', 31, 30, 0], ['NU42', 'NU30', 27, 29, 0], ['NU42', 'US1-TAMARUGO/SBL', 92, 108, 0], ['NU43', 'NU42', 59, 53, 0], ['NU43', 'US2-4335/OXB', 47, 57, 0], ['NU43', 'UI1-INPITUJINA/EST', 49, 70, 0], ['NU43', 'NU48', 59, 74, 0], ['NU44', 'NU40', 15, 21, 1], ['NU44', 'NU14', 12, 12, 0], ['NU44', 'NU39', 29, 23, 0], ['NU45', 'NU15', 135, 191, 0], ['NU45', 'UF0-4060-720-1', 70, 98, 0], ['NU46', 'NU08', 91, 99, 0], ['NU46', 'NU17', 52, 44, 0], ['NU47', 'US1-PEBBLE/SBH', 14, 17, 0], ['NU47', 'NU17', 108, 120, 0], ['NU47', 'US1-PEBBLE/EST', 39, 49, 0], ['NR1059', 'RF10A-4165-1006-1', 37, 34, 0], ['NU48', 'NU43', 66, 59, 0], ['NU48', 'NU65', 56, 88, 0], ['NU49', 'NU50', 20, 27, 0], ['NU49', 'NU60', 97, 122, 0], ['NU50', 'NU49', 22, 27, 0], ['NU50', 'NU59', 47, 59, 0], ['NU51', 'NU59', 44, 53, 0], ['NU51', 'NU61', 55, 67, 0], ['NU52', 'NU61', 76, 56, 0], ['NU52', 'NU62', 97, 110, 0], ['NU53', 'NU63', 141, 89, 0], ['NU53', 'NU64', 55, 65, 0], ['NU53', 'UB1-DRENBASAL/EST', 42, 54, 0], ['NU53', 'NU55', 59, 72, 0], ['NU55', 'NU66', 74, 87, 0], ['NU55', 'NU53', 65, 49, 0], ['NU56', 'NU28', 71, 77, 1], ['NU56', 'NU57', 37, 32, 0], ['NU57', 'NU56', 60, 64, 1], ['NU57', 'NU26', 38, 49, 0], ['NU57', 'NU27', 71, 93, 0], ['NU58', 'NU41', 42, 31, 0], ['NU58', 'NU71', 22, 50, 0], ['NU58', 'CO01', 7, 6, 0], ['NU59', 'NU50', 51, 61, 0], ['NU59', 'NU51', 40, 59, 0], ['NU60', 'NU04', 65, 81, 0], ['NU60', 'NU49', 62, 56, 0], ['NU60', 'NU30', 75, 89, 0], ['NU61', 'NU51', 97, 73, 0], ['NU61', 'NU52', 39, 55, 0], ['NU62', 'NU52', 114, 111, 0], ['NU62', 'NU63', 89, 110, 0], ['NU63', 'NU62', 101, 106, 0], ['NU63', 'NU53', 74, 91, 0], ['NU64', 'NU53', 46, 59, 0], ['NU64', 'NU73', 31, 40, 0], ['NU65', 'NU48', 39, 27, 0], ['NU65', 'NU77', 35, 43, 0], ['NU65', 'UB1-4360TRANQUE/EST', 264, 186, 0], ['NU66', 'NU55', 74, 57, 0], ['NU66', 'NU67', 140, 135, 0], ['NU67', 'NU66', 137, 117, 0], ['NU67', 'NU68', 58, 112, 0], ['NU67', 'UB1-EMPTRANQUE1/EST', 50, 70, 0], ['NU67', 'UB1-MUROTRANQUE/EST', 31, 38, 0], ['NU67', 'UB1-TRANQUEMURO/EST', 34, 40, 0], ['NU68', 'NU67', 56, 53, 0], ['NU68', 'NU69', 99, 88, 0], ['NU68', 'UB1-TRANQUEDROP/EST', 17, 22, 0], ['NU68', 'UB1-GTAMTRANQUE/EST', 17, 15, 0], ['NU62', 'UB1-ESPESADORHRT/EST', 90, 156, 0], ['NU69', 'NU68', 114, 105, 0], ['NU69', 'NU70', 66, 58, 0], ['NU70', 'NU69', 91, 81, 0], ['NU70', 'NU72', 54, 60, 0], ['NU70', 'UB1-DESATRANQUE/EST', 35, 34, 0], ['NU71', 'NU40', 63, 37, 0], ['NU71', 'US1-4365/SME', 13, 33, 0], ['NU71', 'US1-4365/OXB', 38, 52, 0], ['NU71', 'US1-PALOMA/SBH', 54, 45, 0], ['NU71', 'NU58', 14, 20, 0], ['NU71', 'US1-CORREA/OXI', 14, 18, 0], ['NU71', 'US1-PALOMAPEBBLE/SBH', 51, 55, 0], ['NU72', 'NU70', 69, 62, 0], ['NU72', 'UB1-EMPTRANQUE2/EST', 22, 25, 0], ['NU72', 'NU78', 270, 208, 0], ['NU73', 'NU64', 32, 39, 0], ['NU73', 'NU74', 91, 118, 0], ['NU74', 'NU73', 91, 116, 0], ['NU74', 'NU75', 68, 86, 0], ['NU72', 'UB1-EMPRESTITO/EST', 122, 131, 0], ['NU75', 'NU74', 68, 86, 0], ['NU75', 'NU76', 126, 160, 0], ['NU75', 'UB1-TRANQUEBERM/EST', 14, 18, 0], ['NU76', 'NU75', 126, 160, 0], ['NU76', 'UB1-TRANQUEESPA/EST', 115, 146, 0], ['NU77', 'NU65', 43, 52, 0], ['NU77', 'UB1-4390/EST', 48, 55, 0], ['NU78', 'NU72', 309, 193, 0], ['NU78', 'NU79', 178, 141, 0], ['NU79', 'NU78', 224, 141, 0], ['NU79', 'UB1-VERTIND/EST', 10, 26, 0], ['NU79', 'UB1-BATEA/EST', 25, 13, 0], ['P1 UJI', 'NU33', 52, 65, 0], ['P2 SEMIMOVIL', 'NR200', 16, 18, 0], ['P3 SEMIMOVIL', 'NR218', 19, 23, 1], ['P3 SEMIMOVIL', 'NR213', 51, 45, 0], ['P4 SEMIMOVIL', 'NR250', 13, 11, 0], ['P4 SEMIMOVIL', 'NR258', 15, 18, 1], ['PLAT BASE UJINA', 'NU29', 28, 18, 0], ['PLAT CANCHA10', 'NU29', 63, 45, 0], ['PLAT CAPELLA', 'NR905', 15, 19, 0], ['PLAT COBRE', 'NR284', 16, 14, 0], ['PLAT INVIERNO', 'NR314', 56, 47, 0], ['PLAT INVIERNO UJINA', 'NU18', 14, 20, 0], ['PLAT LLAMA 1', 'NR299', 57, 80, 0], ['PLAT LLAMA 2', 'NR271', 45, 57, 0], ['PLAT PUMA 1', 'NR267', 16, 28, 0], ['PLAT PUMA 2', 'NR246', 31, 68, 0], ['PRETILES', 'TALLER ROSARIO', 18, 14, 0], ['PROY. CRUZ', 'NR249', 4, 5, 0], ['PROY. CRUZ', 'NR309', 41, 58, 1], ['R CAPELLA CORTA', 'NR817', 36, 41, 1], ['R CAPELLA CORTA', 'NR1061', 23, 23, 0], ['RAMPA 4600', 'NR1024', 30, 41, 0], ['RAMPA 4600', 'NR1023', 531, 749, 0], ['RAMPA CAPELLA', 'NR1015', 180, 219, 0], ['RAMPA CAPELLA', 'NR1045', 31, 29, 0], ['RAMPA FASE11', 'NR1101', 189, 175, 0], ['RAMPA FASE11', 'NR1102', 18, 15, 0], ['RAMPA NORTE', 'NR641', 37, 43, 1], ['RAMPA NORTE', 'NR643', 162, 260, 0], ['RAMPA TUNEL', 'NR640', 22, 32, 1], ['RAMPA TUNEL', 'NR637', 78, 73, 0], ['RB1-ALPACA2/EST', 'NR350', 13, 17, 1], ['RB1-ALPACA2/EST', 'NR351', 10, 13, 0], ['RB1-CAMINOCHANCA/EST', 'NR363', 10, 10, 0], ['RB1-CHINCHILLA/EST', 'NR233', 21, 26, 0], ['RB1-FALLAGRIS/EST', 'NR1051', 11, 13, 0], ['RB1-FLAMENCO/EST', 'NR337', 33, 48, 0], ['RB1-GUANACOLARGO/EST', 'NR214', 84, 105, 1], ['RB1-GUANACOLARGO/EST', 'NR210', 34, 43, 0], ['RB1-GUANACOMEDIO/EST', 'NR240', 8, 11, 1], ['RB1-GUANACOMEDIO/EST', 'NR230', 34, 44, 0], ['RB1-GUANACOSUP/EST', 'NR383', 49, 60, 0], ['RB1-HUINQUICORTO/EST', 'NR230', 9, 11, 0], ['RB1-HUINQUINEUTR/EST', 'NR230', 16, 18, 1], ['RB1-PATIOPALA/EST', 'NR328', 18, 17, 0], ['RB1-PUMA/EST', 'NR318', 39, 43, 0], ['RB1-PUMA4600/EST', 'NR346', 10, 12, 1], ['RB1-PUMA4600/EST', 'NR378', 11, 13, 0], ['RB1-PUMANEUTRO/EST', 'NR243', 30, 35, 0], ['RB1-RAMPALIRIMA/EST', 'NR352', 9, 10, 0], ['RB1-RAMPAZORRO/EST', 'NR290', 34, 39, 0], ['RB1-ZORRO/EST', 'NR390', 40, 49, 1], ['RB1-ZORRO/EST', 'NR389', 35, 49, 0], ['RB1-ZORRO4675/EST', 'NR245', 72, 110, 0], ['RB2-ALPACA/EST', 'NR365', 22, 26, 1], ['RB2-ALPACA/EST', 'NR360', 38, 50, 0], ['RB2-ALPACACORTO/EST', 'NR345', 44, 55, 1], ['RB2-ALPACACORTO/EST', 'NR344', 61, 70, 0], ['NR1214', 'NR1213', 37, 47, 0], ['RB2-PUMA2/EST', 'NR384', 82, 132, 0], ['RB3-NEUTRO/EST', 'NR243', 14, 18, 0], ['RF12-4615-1233-1', 'NR1209', 25, 51, 0], ['RF10B-4180-1005-1', 'NR1062', 61, 144, 0], ['RF12-4600-1206-1', 'NR120', 39, 56, 0], ['RF11-4300-1101-1', 'NR1130', 28, 60, 0], ['UB1-TRANQUECORON/EST', 'NU74', 18, 23, 0], ['NR1205', 'NR1200', 77, 75, 0], ['RF12-4615-1237-1', 'NR1209', 55, 66, 0], ['NR275', 'NR1206', 22, 28, 0], ['NR1038', 'NR1059', 17, 22, 0], ['RF9B-4030-904-1', 'NR921', 67, 97, 0], ['RI1-INPITFASE6/SME', 'NR605', 15, 18, 0], ['RI10A-INPIT/EST', 'NR1062', 75, 106, 0], ['RI10A-INPIT/SAL', 'NR1062', 22, 21, 0], ['RI10A-INPIT/SBH', 'NR1062', 19, 21, 0], ['RF10B-4195-1005-1', 'NR1010', 17, 21, 0], ['RI10A-INPIT/SME', 'NR1042', 8, 10, 0], ['RI10A-INPITGRIS/EST', 'NR1051', 13, 16, 0], ['RI10A-SOBRECARGA/EST', 'NR1010', 10, 12, 0], ['RI10A-SOBRECARGA/SAL', 'NR1062', 8, 9, 0], ['RI10A-SOBRECARGA/SME', 'NR1042', 8, 10, 0], ['RI10B-INPIT/EST', 'NR1010', 7, 9, 0], ['RI10B-INPIT/SAL', 'NR1042', 7, 9, 0], ['RI10B-INPIT/SBH', 'NR1062', 20, 22, 0], ['NR1063', 'RI10B-INPIT/SBL', 11, 14, 0], ['RI10B-INPIT/SME', 'NR1010', 13, 11, 0], ['RI10B-SOBRECARGA/EST', 'NR1042', 10, 12, 0], ['RI11-INPIT/EST', 'NR1117', 10, 11, 0], ['RI11-INPIT/SAL', 'NR1117', 7, 9, 0], ['RI11-INPIT/SBH', 'NR1110', 11, 14, 0], ['RI11-INPIT/SBL', 'NR1110', 8, 9, 0], ['RI11-INPIT/SME', 'NR1110', 15, 19, 0], ['RI11-SOBRECARGA/EST', 'NR1110', 11, 14, 0], ['RI11-SOBRECARGA/SBH', 'NR1117', 11, 14, 0], ['RI12-INPIT/EST', 'NR1207', 24, 27, 0], ['RI12-INPIT/OXI', 'NR1207', 21, 27, 0], ['RI12-SOBRECARGA/EST', 'NR1207', 23, 29, 0], ['RI8B2-INPIT/SAL', 'NR605', 10, 11, 0], ['RI8B2-INPIT/SME', 'NR605', 7, 7, 0], ['RI9B-INPIT/SAL', 'NR912', 5, 6, 0], ['RI9B-INPIT/SME', 'NR912', 10, 8, 0], ['RI9B-SOBRECARGA/SAL', 'NR912', 5, 7, 0], ['RS1-4550/SBL', 'NR388', 85, 105, 1], ['RS1-4550/SBL', 'NR387', 65, 96, 0], ['RS1-4600/MIX', 'NR231', 21, 23, 0], ['RS1-4600/OXA', 'NR231', 38, 48, 0], ['RS1-4600/OXB', 'NR231', 31, 39, 0], ['RS1-AGUILA/SAA', 'NR325', 34, 40, 0], ['RS1-CHINCHILLA/SME', 'NR233', 69, 88, 0], ['RS1-GAVIOTA/SAL', 'NR284', 24, 49, 0], ['RS1-LIRIMA/MIX', 'NR332', 69, 78, 0], ['RS1-PACHICA/OXI', 'NR283', 211, 99, 0], ['RS1-PARINA/SME', 'NR256', 43, 62, 0], ['RS1-PEJERREY/MIX', 'NR215', 96, 45, 0], ['RS1-STOCKCR04/SME', 'NR311', 13, 20, 0], ['RS1-SURI/SBH', 'NR291', 12, 13, 0], ['RS1-SURI/SBH', 'NR349', 11, 14, 0], ['RS1-TAGUA/SAL', 'NR257', 26, 34, 0], ['RS2-4550/SBL', 'NR256', 17, 20, 0], ['RS2-4600/MIX', 'NR246', 32, 29, 0], ['RS2-4600/OXA', 'NR211', 21, 26, 0], ['RS2-4600/OXB', 'NR231', 49, 39, 0], ['RS2-FLAMENCO/SME', 'NR209', 44, 74, 0], ['RS2-GOLONDRINA/SME', 'NR325', 54, 62, 0], ['RS2-STOCKCR04/SME', 'NR311', 17, 31, 0], ['RS3-4600/MIX', 'NR295', 18, 22, 0], ['RS3-4600/OXB', 'NR246', 20, 19, 0], ['RS3-CARPA/OXA', 'NR227', 29, 36, 0], ['RS3-JUREL/MIX', 'NR212', 34, 44, 0], ['RS3-SALMON/OXA', 'NR239', 56, 51, 0], ['RS3-TOYO/LIX', 'NR239', 38, 47, 0], ['RS4-4600/OXB', 'NR231', 27, 54, 0], ['RS4-CONDOR/SME', 'NR325', 35, 33, 0], ['RS5-4600/OXB', 'NR246', 17, 24, 0], ['SB1-NANDU/EST', 'NR706', 20, 27, 0], ['SI1-ROSARIOSUR/EST', 'NR706', 22, 26, 0], ['STEPOUT4530', 'NR1037', 58, 72, 0], ['STEPOUT4530', 'NR1039', 76, 101, 0], ['TALLER BAYLAC', 'NR314', 77, 101, 0], ['TALLER BAYLAC', 'NR319', 47, 52, 1], ['TALLER ROSARIO', 'NR348', 12, 15, 0], ['TALLER ROSARIO', 'PRETILES', 11, 11, 0], ['TALLER ROSARIO', 'NAVE18', 7, 9, 0], ['TALLER ROSARIO', 'NAVE17', 6, 8, 0], ['TALLER ROSARIO', 'NAVE16', 6, 7, 0], ['TALLER ROSARIO', 'NAVE15', 5, 7, 0], ['TALLER ROSARIO', 'NAVE14', 8, 6, 0], ['TALLER ROSARIO', 'NAVE13', 5, 6, 0], ['TALLER ROSARIO', 'NAVE12', 5, 6, 0], ['TALLER ROSARIO', 'NAVE11', 5, 6, 0], ['TALLER ROSARIO', 'NAVE10', 5, 8, 0], ['TALLER ROSARIO', 'NAVE01', 7, 8, 0], ['TALLER ROSARIO', 'NAVE02', 6, 7, 0], ['TALLER ROSARIO', 'NAVE03', 16, 6, 0], ['TALLER ROSARIO', 'NAVE04', 4, 5, 0], ['TALLER ROSARIO', 'NAVE05', 4, 5, 0], ['TALLER ROSARIO', 'NAVE06', 4, 5, 0], ['TALLER ROSARIO', 'NAVE07', 4, 5, 0], ['TALLER ROSARIO', 'NAVE08', 8, 6, 0], ['TALLER ROSARIO', 'NAVE09', 6, 7, 0], ['TALLER UJINA', 'NU16', 45, 55, 0], ['UB1-4360TRANQUE/EST', 'NU65', 182, 145, 0], ['UB1-4390/EST', 'NU77', 38, 45, 0], ['UB1-DRENBASAL/EST', 'NU53', 48, 54, 0], ['UB1-BATEA/EST', 'NU79', 10, 28, 0], ['UB1-DESATRANQUE/EST', 'NU70', 35, 35, 0], ['UB1-ESPESADORHRT/EST', 'NU62', 64, 98, 0], ['UB1-EMPTRANQUE1/EST', 'NU67', 66, 76, 0], ['UB1-EMPTRANQUE2/EST', 'NU72', 17, 19, 0], ['RF10A-4165-1006-1', 'NR1059', 33, 40, 0], ['UB1-GTAMTRANQUE/EST', 'NU68', 27, 30, 0], ['UB1-MUROTRANQUE/EST', 'NU67', 43, 45, 0], ['UB1-PEBBLE/EST', 'NU17', 51, 54, 0], ['UB1-TRANQUEBERM/EST', 'NU75', 14, 18, 0], ['UB1-EMPRESTITO/EST', 'NU72', 127, 121, 0], ['UB1-TRANQUEDROP/EST', 'NU68', 24, 26, 0], ['UB1-TRANQUEESPA/EST', 'NU76', 115, 146, 0], ['UB1-TRANQUEMURO/EST', 'NU67', 41, 48, 0], ['UB1-VERTIND/EST', 'NU79', 25, 12, 0], ['UF0-4060-720-1', 'NU45', 92, 106, 0], ['UI1-INPITUJINA/EST', 'NU43', 53, 49, 0], ['UI1-INPITUJINA/SBH', 'NU30', 32, 40, 0], ['US1-4210/MIX', 'NU09', 66, 72, 0], ['US1-4210/SBL', 'NU09', 49, 68, 1], ['US1-4330/OXA', 'NU29', 46, 56, 0], ['US1-4330/OXB', 'NU29', 47, 59, 0], ['US1-4335/MIX', 'NU29', 102, 110, 1], ['US1-4335/OXB', 'NU29', 94, 103, 0], ['US1-4365/OXB', 'NU71', 52, 59, 0], ['US1-4365/SME', 'NU71', 26, 18, 0], ['US1-4460/MIX', 'NU38', 16, 22, 0], ['US1-CHACHACOMA/SME', 'NU33', 69, 90, 0], ['US1-CORREA/OXI', 'NU71', 14, 17, 0], ['US1-JOTE/SAL', 'NU04', 44, 54, 0], ['US1-LLARETA/SBH', 'NU30', 28, 35, 0], ['US1-PALOMA/SBH', 'NU71', 97, 126, 0], ['US1-PALOMAPEBBLE/SBH', 'NU71', 102, 94, 0], ['US1-PEBBLE/EST', 'NU47', 51, 60, 0], ['US1-PEBBLE/SBH', 'NU47', 13, 18, 0], ['US1-TAMARUGO/SBA', 'NU30', 160, 194, 0], ['US1-TAMARUGO/SBH', 'NU30', 53, 65, 0], ['US1-TAMARUGO/SBL', 'NU42', 70, 87, 0], ['US2-4335/MIX', 'NU30', 149, 172, 0], ['US2-4335/OXB', 'NU43', 40, 50, 0], ['US2-4335/SBH', 'NU30', 157, 186, 0], ['WASH SHOP', 'NR314', 98, 74, 0], ['RB2-GUANACOCORTO/EST', 'NR362', 65, 121, 0], ['NR362', 'RB2-GUANACOCORTO/EST', 47, 59, 0], ['NR1208', 'NR275', 32, 41, 0], ['NR275', 'NR1208', 31, 32, 0], ['NR1010', 'NR1056', 37, 46, 1], ['NR1056', 'NR1010', 50, 56, 1], ['NR1063', 'RF10B-4180-1005-1', 26, 30, 0], ['RF10B-4180-1005-1', 'NR1063', 33, 37, 0], ['NR284', 'RS2-GAVIOTA/SAL', 19, 39, 0], ['RS2-GAVIOTA/SAL', 'NR284', 17, 21, 0], ['NR1056', 'NR1038', 40, 55, 1], ['NR1038', 'NR1056', 42, 53, 1], ['NR1059', 'NR1063', 39, 47, 0], ['NR1063', 'NR1059', 44, 43, 0], ['NR1062', 'NR1063', 50, 43, 0], ['NR1063', 'NR1062', 64, 52, 0], ['RS1-FLAMENCO/SME', 'NR209', 35, 25, 0], ['NR209', 'RS1-FLAMENCO/SME', 22, 59, 0], ['RF12-4600-1208-1', 'NR1213', 50, 65, 0], ['NR1213', 'RF12-4600-1208-1', 46, 55, 0]]

#Datos arcos del grafo
LocStart = [Arcos[i][0] for i in range(0,len(Arcos))]    #Origen arco
LocEnd = [Arcos[i][1] for i in range(0,len(Arcos))]      #Destino arco
TimeEmpty = [Arcos[i][2] for i in range(0,len(Arcos))]   #Tiempo vacío arco
TimeLoad = [Arcos[i][3] for i in range(0,len(Arcos))]    #Tiemp cargado arco
Closed = [Arcos[i][4] for i in range(0,len(Arcos))]      #Camino cerrado (1 o 0

#Crear grafo completo desde "tabla" MTV
Nodos = pd.unique(LocStart+LocEnd) 
Grafo = nx.DiGraph()
Grafo.add_nodes_from(Nodos)

for i in range(0,len(Arcos)):
    Grafo.add_edge(LocStart[i], LocEnd[i], time_empty = TimeEmpty[i], time_load = TimeLoad[i], closed = Closed[i])

# EJecutar algoritmo Floyd_Warshall
predecessors, distance = nx.floyd_warshall_predecessor_and_distance(Grafo, weight='time_empty')
predecessors_loaded, distance_loaded = nx.floyd_warshall_predecessor_and_distance(Grafo, weight='time_load')
#Se construye diccionario (NodoInicio,NodoTérmino)->(lista camino minimo, Lista tiempos acumulados)
caminosMinimosYTiemposAcumulados = {}

for nodoInicio in Nodos:
    for nodoTermino in Nodos:
        camino = nx.reconstruct_path(nodoInicio,nodoTermino,predecessors)
        tiempos = []
        tiempoAcumulado = 0
        tiempos.append(tiempoAcumulado)
        for i in range(0,len(camino)-1):
            tiempoAcumulado += distance[camino[i]][camino[i+1]]
            tiempos.append(tiempoAcumulado)
        caminosMinimosYTiemposAcumulados[(nodoInicio,nodoTermino)] = (camino,tiempos)


# MTV

In [9]:
MTV = {}
for o in distance.keys():
    MTV[o] = {}
    for d in distance[o].keys():
        MTV[o][d] = {}
        if o == d:
            MTV[o][d][0] = 0
            MTV[o][d][1] = 0
            
        else:
            try:
                MTV[o][d][0] = distance[o][d] 
                MTV[o][d][1] = distance_loaded[o][d]
        
            except:#Si el grafo no es conexo relleno
                print((o, d))
        

eliminar = []
for nodo in Polvorazos:
    if nodo in list(MTV.keys()):
        pass
    else:
        eliminar.append(nodo)

for i in eliminar:
    Polvorazos.remove(i)
    Nodos_Mineral.remove(i)
    
eliminar = []
for nodo in SoB:
    if nodo in list(MTV.keys()):
        pass
    else:
        #print(nodo)
        eliminar.append(nodo)

for i in eliminar:
    SoB.remove(i)
    

# Clases
## Palas

In [10]:
class Pala:
    def __init__(self, ID, capacidad, ubicacion, media_recuperacion, media_falla, velocidad, tiempo_aculatado, min_carga, moda_carga, max_carga, pat):
        self.ubicacion = ubicacion #Nodo en que esta ubicada la pala
        self.capacidad = capacidad #Capacidad nominal de excavacion
        self.ID = ID #ID pala
        self.fila = deque() #camiones en fila para se atendidos
        
        self.espacios_aculatamiento = [0, 0] #camiones aculatados 
        self.orden_llegada = deque() #orden de llegada al aculatamiento
        #self.llegada_ultimo_camion = 0
        self.hora_ultima_asignacion = 0 #Lj
    
        self.horas_eventos = {"Recuperacion":99999999, "Caida":9999999, "Inicio Movimiento":99999999, "Fin Movimiento":99999999, "Carga":999999} 
        self.proximo_evento = None #Nombre del evento mas proximo
        #self.toneladas_sacadas = {} #Diccionario de tuplas con toneladas sacadas por posicion/nodo y ley
        self.TPH = {} #Diccionario con los resultados del PL
        
        self.media_recuperacion = media_recuperacion
        self.media_falla = media_falla
        self.velocidad = velocidad
        
        self.tiempo_aculatado = tiempo_aculatado
        
        self.min = min_carga
        self.moda = moda_carga
        self.max = max_carga
        self.capacidad_nominal = 9000
        self.hora_siguiente = 999999999
        self.disponibilidad = True #True: Disponible, False: Ocupada
        self.estado = "Funcionando" #Funcionando o Mala
        self.toneladas_sacadas = []#Lleva el registro de las excavaciones: Lugar, toneladas y destino
        self.camiones_asignados = []#(reloj, camion)
        self.tasa_atencion = 3/(self.min+self.moda+self.max)
        self.ley_de_mineral = 0.98
        #Pronostico de la la ley: [toneladas, ley, clasificacion(mineral o esteril)] 1:mineral 0 esteril
        #self.pronostico_avance_ley = [[10000, random.uniform(0.8, 1.2), 1] for i in range(50)]
        self.pronostico_avance_ley = pat
        self.eventos_programados = [[]]
        
    def caida(self, Reloj):#Completar comn PL y con Dinamico
        self.estado = "Mala"
        self.disponibilidad = False
        self.horas_eventos["Caida"] = 9999 * Reloj
        self.horas_eventos["Recuperacion"] = Reloj + self.tiempo_recuperacion()
        if self.horas_eventos["Recuperacion"] > self.horas_eventos["Inicio Movimiento"]:
            self.horas_eventos["Inicio Movimiento"] = self.horas_eventos["Recuperacion"] + 0.0001 #Postergo el movimiento hasta que se arregle la pala
        self.proximo_evento = "Recuperacion"
        return None
    
    def recuperacion(self, Reloj):
        self.estado = "Funcionando"
        self.disponibilidad = True
        self.horas_eventos["Recuperacion"] = 9999 * Reloj
        self.horas_eventos["Caida"] = Reloj + self.tiempo_falla()
        self.proximo_evento = min(self.horas_eventos, key=self.horas_eventos.get)
        self.hora_siguiente = self.horas_eventos[self.proximo_evento]
        return None
    
    def disponible(self):
        for espacio in self.espacios_aculatamiento:
            if espacio != 0:
                return True
        return False
    
    def fin_movimiento(self, Reloj, Lista_movimientos_agendados):
        pass
       
    def tiempo_recuperacion(self):
        return(np.random.exponential(self.media_recuperacion))
    
    def tiempo_falla(self):
        return(np.random.exponential(self.media_falla))
    
    def hora_proximo_evento(self):
        self.hora_siguiente = min(self.horas_eventos.values())
        self.proximo_evento = min(self.horas_eventos, key=self.horas_eventos.get)
        return(self.hora_siguiente)
    
    def nombre_proximo_evento(self):
        self.proximo_evento = min(self.horas_eventos, key=self.horas_eventos.get)
        return(self.proximo_evento)
    
    def duracion_atencion(self):
        return(np.random.triangular(self.min, self.moda, self.max))
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        

## Camiones

In [11]:
class Camion:
    def __init__(self, ID, size):
        self.ID = ID
        self.Nombre = "C"+str(ID)
        self.destino = comienzo_camion
        self.origen = comienzo_camion
        self.almuerzo = 0 #0: No almorzo, 1 primera colacion, 2 segunda colacion (solo aplica turno B)
        self.hora_siguiente = 0
        self.descargas = {} #Diccionario que almacena la historia de las descargas: key = (Origen, Destino)| value = toneladas
        self.carga = 0
        self.estado = "Descargando" #Viajando, Aculatandose, Cargando, Descargando, Almorzando, En fila
        self.OD = None
        self.hora_salida = 0
        self.cargado = 0 #1 si esta cargado, 0 en caso contrario
        self.size = size
        
    def descargar(self, nuevo_destino, Reloj, TdV):
        self.toneladas = 0
        self.origen = self.destino
        self.destino = nuevo_destino
        #if self.carga > 0:
        #    self.descargas[(self.origen, destino)] = self.carga 
        self.carga = 0
        self.cargado = 0
        self.hora_salida = Reloj
        self.estado = "Viajando"
        self.hora_siguiente = Reloj + TdV
        return None
    
    def redirigir_viaje(self, nodo_inicio, Reloj, TdV):
        self.estado = "Viajando"
        self.origen = nodo_inicio
        self.destino = self.OD[0]
        self.carga = 0
        self.cargado = 0
        self.hora_salida = Reloj
        self.hora_siguiente = Reloj + TdV
        
    def cargar(self, toneladas_cargadas, nuevo_destino):
        self.origen = self.destino
        self.destino = nuevo_destino
        self.carga = toneladas_cargadas
        return None
    
    #def asignar_destino(self, Reloj, nuevo_destino, MTV):
    #    self.destino = nuevo_destino
    #    self.actualizar_estado("Viajando", Reloj + MTV[self.origen][self.destino])
    
    def actualizar_estado(self, Estado, Hora):
        self.estado = Estado 
        self.hora_siguiente = Hora
        
    def nuevo_viaje(self, Estado, Hora):
        self.estado = Estado 
        self.hora_siguiente = Hora
        self.origen = self.OD[0]
        self.destino = self.OD[1]
        
    def reasignacion(self, OD, siguiente, nodo_inicio):
        self.origen = nodo_inicio
        self.destino = OD[0]
        self.OD= OD
        self.estado = "Viajando"
        self.hora_siguiente = siguiente 
        
    def comenzar_almuerzo(self, destino, reloj, TV, tiempo_colacion, almuerzo):
        self.origen = self.destino
        self.destino = destino
        self.estado = "Almorzando"
        self.hora_siguiente = Reloj + TV + tiempo_colacion
        self.almuerzo = almuerzo
        self.carga = 0
        self.cargado = 0
        
    def __str__(self):
        return str(self.Nombre)
    def __unicode__(self):
        return str(self.Nombre)
    def __repr__(self):
        return str(self.Nombre)


## Stock - Botadero

In [12]:
class Stock:
    
    def __init__(self, ID, espacios, ubicacion, tiempo_atencion, tiempo_aculatado):
        self.ID = ID
        self.fila = deque()
        self.espacios_aculatamiento = [0 for i in range(espacios)]
        self.ubicacion = ubicacion
        self.tiempo_atencion = tiempo_atencion
        self.hora_siguiente = 99999999
        self.tiempo_aculatado = tiempo_aculatado 
        self.capacidad = 99999999999
        self.disponibilidad = True
    
    def avanzar_fila(self):
        camion = self.fila.popleft()
        return camion
    
    def duracion_atencion(self):
        return(self.tiempo_atencion)
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        
   

## Chancador

In [13]:
class Chancador:
    
    def __init__(self, ID, ubicacion, media_falla, media_recuperacion, tiempo_aculatado, min_carga, moda_carga, max_carga):
        self.ID = ID
        self.ubicacion = ubicacion
        self.espacios_aculatamiento = [0, 0]
        self.fila = deque()
        self.media_falla = media_falla
        self.media_recuperacion = media_recuperacion
        self.tiempo_aculatado = tiempo_aculatado
        self.min = min_carga
        self.moda = moda_carga
        self.max = max_carga
        self.hora_siguiente = 999999999
        self.nivel_actual = 0
        self.tasa_procesamiento = 175 #toneladas/minuto
        self.ultima_descarga = 0 #hora en que se realizo la ultima descarga
        self.tiempo_atencion = (self.min + self.moda + self.max)/3
        self.capacidad = 10500
        self.disponibilidad = True
        self.DMPP = {} #Descargas de mineral por pala
        self.stock_pile = 'SP1'
    
    def caida(self, Reloj):
        self.disponibildad = False
        self.hora_siguiente = Reloj + self.tiempo_recuperacion
        
    def recuperacion(self, Reloj):
        self.disponibilidad = True
        self.hora_siguiente = Reloj + self.tiempo_falla
        
    def tiempo_recuperacion(self):
        return(np.random.exponential(self.media_recuperacion))
    
    def tiempo_falla(self):
        return(np.random.exponential(self.media_falla))
    
    def hora_proximo_evento(self):
        return(min(horas_eventos.values()))
    
    def duracion_atencion(self):
        return(np.random.triangular(self.min, self.moda, self.max))
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        
    

# Inicialización 

In [14]:
#Se evalúa que palas estarán operativas y en que condición y se generan las instancias 
#'''
#L_Entidades = []
##Esteril: 303
##Mineral: 293
#Capacidades = [303, 293]
#D_Palas = {}
#D_Entidades = {}
#for pala in InfoPalas:
#    u = np.random.uniform(0,1)
#    
#    if 0 <= u and u < pala[3]:
#        pass
#    
#    elif pala[3] <= u and u < 1:#pala[4]
#        D_Palas[pala[0]] = Pala(pala[0], random.choice(Capacidades), random.choice(Polvorazos), pala[2], pala[1], "velocidad", \
#                                pala[8], pala[5], pala[6], pala[7])
#        L_Entidades.append(D_Palas[pala[0]])
#        D_Entidades[pala[0]] = D_Palas[pala[0]]
#       
#    else:
#        #D_Palas[pala[0]] = Pala(pala[0], "TONELADAS X CARGA", random.choice(Polvorazos), pala[2], pala[1], "velocidad", \
#        #                        pala[8], pala[5], pala[6], pala[7]) 
#        #D_Palas[pala[0]].recuperacion(0)#Se inicializa el tiempo de la siguiente falla
#        #print(pala[0], D_Palas[pala[0]].hora_siguiente)
#        #L_Entidades.append(D_Palas[pala[0]])
#        #D_Entidades[pala[0]] = D_Palas[pala[0]]
#        pass
#        
##Se inicializan los camiones
#D_Camiones = {}
#for i in range(total_camiones):
#    D_Camiones[i] = Camion(i, size_camion)
#    L_Entidades.append(D_Camiones[i])
#    D_Entidades[i] = D_Camiones[i]
#
#
#    
##Se inicializan los stocks/botaderos
#D_Stocks = {}
#for stock in SoB:
#    D_Stocks[stock] = Stock(stock, espacios, stock, tiempo_atencion_stock, tiempo_aculatamiento)
#    L_Entidades.append(D_Stocks[stock])
#    D_Entidades[stock] = D_Stocks[stock]
##print("Stocks y Botaderos :", D_Stocks)    
#
##Se inicializa el Chancador
#D_Chancador ={}
#for chancador in InfoChancador:
#    D_Chancador[chancador[0]] = Chancador(chancador[0], "CS03", chancador[1], chancador[2], chancador[8], \
#                                          chancador[5], chancador[6], chancador[7])
#    L_Entidades.append(D_Chancador[chancador[0]])
#    D_Entidades[chancador[0]] = D_Chancador[chancador[0]]
#    
##print("Chancador: ", D_Chancador)
#
#
#L_Entidades.sort(key=lambda x: x.hora_siguiente, reverse=False)
#'''

In [15]:
L_Entidades = []
Destinos = []
Palas = []
D_Palas = {}
D_Entidades = {}
contador = 0

for pala in InfoPalas:
    if Polvorazos[contador] in Nodos_Mineral:
        D_Palas[pala[0]] = Pala(pala[0], 303, Polvorazos[contador], pala[2], pala[1], "velocidad", \
                            pala[8], pala[5], pala[6], pala[7], [[10000, random.uniform(0.8, 1.2), 1] for i in range(50)])
    if Polvorazos[contador] in Nodos_Esteril:
        D_Palas[pala[0]] = Pala(pala[0], 293, Polvorazos[contador], pala[2], pala[1], "velocidad", \
                            pala[8], pala[5], pala[6], pala[7], [[10000, random.uniform(0.8, 1.2), 0] for i in range(50)])
        
    L_Entidades.append(D_Palas[pala[0]])
    D_Entidades[pala[0]] = D_Palas[pala[0]]
    Palas.append(D_Palas[pala[0]].ID)
    contador+=1
       
D_Camiones = {}
for i in range(total_camiones):
    D_Camiones[i] = Camion(i, size_camion)
    L_Entidades.append(D_Camiones[i])
    D_Entidades[i] = D_Camiones[i]  

D_Chancador ={}
for chancador in InfoChancador:
    D_Chancador[chancador[0]] = Chancador(chancador[0], nodo_chancador, chancador[1], chancador[2], chancador[8], \
                                          chancador[5], chancador[6], chancador[7])
    L_Entidades.append(D_Chancador[chancador[0]])
    D_Entidades[chancador[0]] = D_Chancador[chancador[0]]
    Destinos.append(D_Entidades[chancador[0]].ID)
    
D_Stocks = {}
indice = 0
for stock in SoB:
    idstock = "S"+str(indice)
    D_Stocks[idstock] = Stock(idstock, espacios, stock, tiempo_atencion_stock, tiempo_aculatamiento)
    L_Entidades.append(D_Stocks[idstock])
    D_Entidades[D_Stocks[idstock].ID] = D_Stocks[idstock]
    Destinos.append(D_Stocks[idstock].ID)
    indice += 1
    

L_Entidades.sort(key=lambda x: x.hora_siguiente, reverse=False)

print("Palas: {}".format(Palas))
print("Camiones: {}".format(D_Camiones))
print("Stocks: {}".format(D_Stocks))
print("Chancador: {}".format(D_Chancador))

Palas: ['PAA', 'PAB', 'PAC', 'PAD']
Camiones: {0: C0, 1: C1, 2: C2, 3: C3, 4: C4, 5: C5, 6: C6, 7: C7, 8: C8, 9: C9, 10: C10, 11: C11, 12: C12, 13: C13, 14: C14, 15: C15, 16: C16, 17: C17, 18: C18, 19: C19, 20: C20, 21: C21, 22: C22, 23: C23, 24: C24, 25: C25, 26: C26, 27: C27, 28: C28, 29: C29, 30: C30, 31: C31, 32: C32, 33: C33, 34: C34, 35: C35, 36: C36, 37: C37, 38: C38, 39: C39}
Stocks: {'S0': S0, 'S1': S1, 'S2': S2}
Chancador: {'CS03': CS03}


In [16]:
Nodos = Nodos_Mineral + Nodos_Esteril + SoB
Nodos.append(nodo_chancador)
Nodos.append(nodo_casino)

# PL

## Inputs modelo

In [17]:
def Inputs_PL(Reloj, total_camiones, size_camion, Palas, D_Entidades, Destinos, Nodos_Fase, L_Stocks, L_Botaderos, Plan_Esteril_Actualizado, Plan_Mineral_Actualizado):
    
    MIT = NTurnos*60*12
    t_global = (Reloj-MIT)/60
    NC = total_camiones
    L = size_camion
    Ttot = 13
    NP = len(Palas)
    NombrePala = [D_Entidades[p].ID for p in Palas]
    LocPala = [D_Entidades[p].ubicacion for p in Palas]
    CP = [D_Entidades[p].capacidad_nominal for p in Palas]
    TC = [1/D_Entidades[p].tasa_atencion for p in Palas]
    PDisp = []
    for p in Palas:
        if D_Entidades[p].estado == "Funcionando":
            PDisp.append(1)
        else:
            PDisp.append(0)
    Ley = [D_Entidades[p].ley_de_mineral for p in Palas]
    ND = len(Destinos)
    NombreDestino = [D_Entidades[s].ID for s in Destinos]
    TD = [D_Entidades[s].tiempo_atencion for s in Destinos]
    CD = [D_Entidades[s].capacidad for s in Destinos]
    DDisp = [D_Entidades[s].disponibilidad*1 for s in Destinos]
    
    cS1 = [i for i in range(len(LocPala)) if LocPala[i] in Nodos_Mineral]
    cS4 = [i for i in range(len(LocPala)) if LocPala[i] in Nodos_Esteril]
    
    cS2 = [i for i in range(len(Destinos)) if D_Entidades[Destinos[i]].__class__.__name__ == "Chancador"]
    cS3 = [i for i in range(len(Destinos)) if D_Entidades[Destinos[i]].ubicacion in L_Stocks]
    cS5 = [i for i in range(len(Destinos)) if D_Entidades[Destinos[i]].ubicacion in L_Botaderos]
    
    NombreFases = [key for key in Nodos_Fase.keys()]
    F = []
    for fase in NombreFases:
        lista = []
        for pala in NombrePala:
            if D_Entidades[pala].ubicacion in Nodos_Fase[fase][0] or D_Entidades[pala].ubicacion in Nodos_Fase[fase][1]:
                lista.append(1)
            else: lista.append(0)
        F.append(lista)
        
    DMPP = [Plan_Mineral_Actualizado[pala] for pala in NombrePala]
    CEPP = [Plan_Esteril_Actualizado[sb] for sb in NombreFases]
    
    
    alphaPM = 0           
    alphaPF = 0           
    alphaSlack = 999999999
    DifMaxPal = 1           
    
    TVR = []
    NDNP = NombreDestino + NombrePala
  
    for i in range(len(NDNP)):
        TVR.append([])
        for j in range(len(NDNP)):
            if (i < ND and j < ND) or (i >= ND and j >= ND):
                #print(NDNP[i], NDNP[j], i, j)
                TVR[i].append(99)
                
            elif (i < ND and j >= ND):
                TVR[i].append(MTV[D_Entidades[NDNP[i]].ubicacion][D_Entidades[NDNP[j]].ubicacion][0]/60)
            else:
                TVR[i].append(MTV[D_Entidades[NDNP[i]].ubicacion][D_Entidades[NDNP[j]].ubicacion][1]/60)
                
    PM0 = [Plan_Mineral[nombre] for nombre in NombrePala] 
    PF0 = [D_Plan_Esteril[sb] for sb in NombreFases]
    return(PF0, PM0, TVR, t_global, NC, L, Ttot, NP, NombrePala, LocPala, CP, TC, PDisp, Ley, ND, NombreDestino, TD, CD, DDisp, cS1, \
           cS4, cS2, cS3, cS5, NombreFases, F, DMPP, CEPP, alphaPM, alphaPF, alphaSlack, DifMaxPal)
    


In [18]:
#PL = {}
#for i in Palas + Destinos:
#    PL[i] = {}
#    for j in Palas + Destinos:
#        if i in Palas and j in Palas:
#            PL[i][j] = 0
#        elif i in Destinos and j in Destinos:
#            PL[i][j] = 0
#        else:
#            PL[i][j] = 2000
#   
NTurnos = 0
PF0, PM0, TVR, t_global, NC, L, Ttot, NP, NombrePala, LocPala, CP, TC, PDisp, Ley, ND, NombreDestino, TD, CD, DDisp, \
cS1, cS4, cS2, cS3, cS5, NombreFases, F, DMPP, CEPP, alphaPM, alphaPF, alphaSlack, DifMaxPal = Inputs_PL(Reloj, total_camiones, size_camion, Palas, D_Entidades, Destinos, Nodos_Fase, L_Stocks, L_Botaderos, Plan_Esteril_Actualizado, Plan_Mineral_Actualizado)

PL = process(TVR, alphaPM, alphaPF, alphaSlack, DifMaxPal, L, NombrePala, ND, NP, PM0, F, PF0, PDisp, TC, CP, TD, CD, DDisp, \
        DMPP, cS1, cS2, cS3, cS4, cS5, NC, t_global, LocPala, NombreDestino, Ley, LeyPromedio, NombreFases)

for value in PL:
    print(PL[value])

Academic license - for non-commercial use only
{'CS03': 0.0, 'S0': 0.0, 'S1': 0.0, 'S2': 0.0, 'PAA': 3404.589197789156, 'PAB': 3404.589197789156, 'PAC': 0.0, 'PAD': 0.0}
{'CS03': 0.0, 'S0': 0.0, 'S1': 0.0, 'S2': 0.0, 'PAA': 0.0, 'PAB': 0.0, 'PAC': 0.0, 'PAD': 0.0}
{'CS03': 0.0, 'S0': 0.0, 'S1': 0.0, 'S2': 0.0, 'PAA': 0.0, 'PAB': 0.0, 'PAC': 0.0, 'PAD': 0.0}
{'CS03': 0.0, 'S0': 0.0, 'S1': 0.0, 'S2': 0.0, 'PAA': 0.0, 'PAB': 0.0, 'PAC': 1702.294598894578, 'PAD': 1702.294598894578}
{'CS03': 3404.589197789156, 'S0': 0.0, 'S1': 0.0, 'S2': 0.0, 'PAA': 0.0, 'PAB': 0.0, 'PAC': 0.0, 'PAD': 0.0}
{'CS03': 3404.589197789156, 'S0': 0.0, 'S1': 0.0, 'S2': 0.0, 'PAA': 0.0, 'PAB': 0.0, 'PAC': 0.0, 'PAD': 0.0}
{'CS03': 0.0, 'S0': 0.0, 'S1': 0.0, 'S2': 1702.294598894578, 'PAA': 0.0, 'PAB': 0.0, 'PAC': 0.0, 'PAD': 0.0}
{'CS03': 0.0, 'S0': 0.0, 'S1': 0.0, 'S2': 1702.294598894578, 'PAA': 0.0, 'PAB': 0.0, 'PAC': 0.0, 'PAD': 0.0}


# Simulación

In [22]:
#Identificamos el siguiente evento que va a ocurrir}
TurnoA = True
TurnoB = False
NTurnos = 0
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

Entidad = L_Entidades[0]
Reloj = Entidad.hora_siguiente

while Reloj < 2*12*60:
    
    Entidad = L_Entidades[0]
    Reloj = Entidad.hora_siguiente
    NTurnos = int(Reloj/(12*60))
    
    if int(Reloj/(12*60))%2 == 0 and TurnoB:
        #Se deben acutualizar todos los estados de almuerzo de los camiones
        for camion in D_Camiones.keys():
            D_Camiones[camion].almuerzo = 0
        TurnoB = False
        TurnoA = True
        
    elif int(Reloj/(12*60))%2 != 0 and TurnoA:
        #Se deben acutualizar todos los estados de almuerzo de los camiones
        for camion in D_Camiones.keys():
            D_Camiones[camion].almuerzo = 0   
        TurnoA = False
        TurnoB = True
    ############################################CAMION#########################################################
    #En caso de que la entidad sea un camion
    if Entidad.__class__.__name__ == "Camion":
        print("Estado del Camion {} a las {}: {}".format(Entidad, Reloj, Entidad.estado))
        print("    Origen: {}".format(Entidad.origen))
        print("    Destino: {}".format(Entidad.destino))
        print("    Hora de Salida: {}".format(Entidad.hora_salida))
        print("    Estado: {}".format(Entidad.estado))
        print("    Almuerzo: {}".format(Entidad.almuerzo))
        print("    Cargado: {}".format(Entidad.cargado))
        print("    Carga: {}\n".format(Entidad.carga))
        
        
        if Entidad.estado == "Almorzando":#Cuando termina de almorzar
            Entidad.estado = "Descargando"#Para que sea despachado en el siguiente if
            
        if Entidad.estado == "Descargando":#Salida de un stock, botadero o chancador
            #Almuerzo turno A
            Entidad.hora_salida = Reloj
            Entidad.cargado = 0
            MIT = NTurnos*60*12 #Minuto en que inicio turno
            Hora_Almuerzo(D_Entidades, Entidad, Reloj, TurnoA, TurnoB, MIT, MTV, AlmuerzoTA, Almuerzo1TB, Almuerzo2TB, nodo_casino)
            
            #En caso contrario, hay que asignarle un nuevo destino corriendo el dinámico
            if Entidad.estado == "Descargando": 
                #Pregunto si hay fila en el stock/botadero/chancador
                try:#Solo en caso de que este en un stock/botadero/chancador
                    Destino = D_Entidades[Entidad.destino]
                    if Destino.__class__.__name__ == "Chancador":
                        Destino.nivel_actual += Entidad.carga
                        Destino.nivel_actual -= Destino.tasa_procesamiento*(Reloj-Destino.ultima_descarga)
                        Destino.ultima_descarga = Reloj
                        Plan_Mineral_Actualizado[Entidad.origen] = max(0, Plan_Mineral_Actualizado[Entidad.origen] - Entidad.carga) 
                       
                    elif Destino.__class__.__name__ == "Stock":
                        for fase in Plan_Esteril_Actualizado.keys():
                            if D_Entidades[Entidad.origen].ubicacion in Nodos_Fase[fase][0]:
                                Plan_Esteril_Actualizado[fase] = max(0, Plan_Esteril_Actualizado[fase] - Entidad.carga)
###############################AGREGAR RESTRICCION DE CAPACIDAD DEL CHANCADOR#####################################   

                    if len(Destino.fila) > 0:
                        #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                        i = Destino.espacios_aculatamiento.index(Entidad)
                        Siguiente = Destino.fila.popleft()
                        Destino.espacios_aculatamiento[i] = Siguiente
                        Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)

                    elif Entidad in Destino.espacios_aculatamiento:
                        i = Destino.espacios_aculatamiento.index(Entidad)
                        Destino.espacios_aculatamiento[i] = 0
                except:
                    pass
                
                nuevo_destino =  dinamico_dispatch(caminosMinimosYTiemposAcumulados, Entidad, camiones_asignables(D_Camiones, D_Entidades), D_Palas, MTV, Reloj, PL, D_Entidades, nodo_casino)   
                Entidad.OD = nuevo_destino #Guardo el resultado del dinamico - par OD
                #Finalizo descarga, actualizo tiempo de sgte evento, y cambio estado a "Viajando", y origen
                if Entidad.destino == nodo_casino:
                    Entidad.descargar(nuevo_destino[0], Reloj, MTV[nodo_casino][D_Entidades[nuevo_destino[0]].ubicacion][Entidad.cargado]) 
                else:
                    Entidad.descargar(nuevo_destino[0], Reloj, MTV[D_Entidades[Entidad.destino].ubicacion][D_Entidades[nuevo_destino[0]].ubicacion][Entidad.cargado]) 

                #Se debe actualizar el atributo Lj de la Pala 
                try:
                    D_Entidades[Entidad.destino].hora_ultima_asignacion = Reloj
                    D_Entidades[Entidad.destino].camiones_asignados.append((Reloj, Entidad))
                except:
                    pass
                
        elif Entidad.estado == "Viajando":
            Destino = D_Entidades[Entidad.destino]
        
            if len(Destino.fila) > 0:#Reviso si hay fila en el destino
                Destino.fila.append(Entidad)#El camion se pone a la fila
                Entidad.actualizar_estado("En fila", Reloj*99)
                #print("Camion {} se pone a la fila {}".format(Entidad, Destino.fila))
            
            if len(Destino.fila) == 0:
                #Se revisa la disponibilidad de espacios de aculatamiento
                if 0 in Destino.espacios_aculatamiento:
                    Entidad.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                    #El camion se ubica en el lugar disponible
                    lugar = 0
                    for espacio in Destino.espacios_aculatamiento:
                        if espacio == 0:
                            Destino.espacios_aculatamiento[lugar] = Entidad
                            if Destino.__class__.__name__ == "Pala":
                                Destino.orden_llegada.append(Entidad)
                            break
                        lugar += 1
                else:
                    Destino.fila.append(Entidad)
                    Entidad.actualizar_estado("En fila", Reloj*99)  
        
        elif Entidad.estado == "Aculatandose":#Termina de aculatarse
            Destino = D_Entidades[Entidad.destino]
            
            if Destino.__class__.__name__ in ["Chancador", "Stock"]:
                Entidad.actualizar_estado("Descargando", Reloj + Destino.duracion_atencion())#Comienza descarga solo
                
            if Destino.__class__.__name__ == "Pala":
                Entidad.actualizar_estado("Cargando", Reloj*99)
                Primero = Destino.orden_llegada.popleft()
                #Reviso si hay alguien mas siendo atendido:
                if Destino.disponibilidad and Entidad == Primero:#Si no hay nadie
                    Destino.horas_eventos["Carga"] = Reloj + Destino.duracion_atencion()
                    Destino.hora_proximo_evento()#Actualizamos la hora del evento siguiente
                    Destino.disponibilidad = False
                else: #Si no hay disponibilidad, se espera a que termine la atencion en el lugar de aculatamiento
                    pass
                Destino.orden_llegada.insert(0, Primero)
                
        print("\nNuevo estado del Camion {} a las {}: {}".format(Entidad, Reloj, Entidad.estado))
        print("    Origen: {}".format(Entidad.origen))
        print("    Destino: {}".format(Entidad.destino))
        print("    Hora de Salida: {}".format(Entidad.hora_salida))
        print("    Estado: {}".format(Entidad.estado))
        print("    Almuerzo: {}".format(Entidad.almuerzo))
        print("    Cargado: {}".format(Entidad.cargado))
        print("    Carga: {}".format(Entidad.carga))
        print("    Siguiente evento: {}\n".format(Entidad.hora_siguiente))
        
    ##########################################PALA#########################################################
    
    if Entidad.__class__.__name__ == "Pala":
        
        print("Estado de la pala {} a las {}".format(Entidad, Reloj))
        print("    Disponibilidad: {}".format(Entidad.disponibilidad))
        print("    Estado: {}".format(Entidad.estado))
        print("    Orden de llegada: {}".format(Entidad.orden_llegada))
        print("    Espacios aculatamiento: {}".format(Entidad.espacios_aculatamiento))
        print("    Fila: {}".format(Entidad.fila))
        print("    Capacidad: {}".format(Entidad.capacidad))
        
        if Entidad.nombre_proximo_evento() == "Carga":#Finaliza la carga del camion en atencion
            #Se busca el camion que termino la atencion
            Camion = Entidad.orden_llegada.popleft()
            #Despacho al camion:
            i = Entidad.espacios_aculatamiento.index(Camion)
            Entidad.espacios_aculatamiento[i] = 0 
            Camion.carga = Entidad.capacidad
            Camion.cargado = 1
            Entidad.toneladas_sacadas.append((Reloj, Entidad.ubicacion, Entidad.capacidad, Camion.Nombre, Camion.OD[1]))
            ########################Cambio en la ley de mineral y capacidad de extraccion###########################
            Entidad.pronostico_avance_ley[0][0] -= Entidad.capacidad
            if Entidad.pronostico_avance_ley[0][0] <= 0:
                Entidad.pronostico_avance_ley.remove(Entidad.pronostico_avance_ley[0])
                Entidad.ley = Entidad.pronostico_avance_ley[0][1]
                if bool(Entidad.pronostico_avance_ley[0][2]):#Si es pala de mineral
                    Entidad.capacidad = Extraccion_Mineral
                else:
                    Entidad.capacidad = Extraccion_Esteril
                #Si el nodo cambia de material se hace la modificacion
                modificar_nodo(Nodos_Fase, Entidad.ubicacion, Entidad.pronostico_avance_ley[0][2])
                Nodos_Mineral, Nodos_Esteril, Polvorazos = getNodos(Nodos_Fase)
            ########################################################################################################
            Camion.nuevo_viaje("Viajando", MTV[D_Entidades[Camion.OD[0]].ubicacion][D_Entidades[Camion.OD[1]].ubicacion][Camion.cargado]+Reloj)
            Camion.hora_salida = Reloj
            Entidad.disponibilidad = True
            Entidad.horas_eventos["Carga"] = Reloj*99
            Entidad.hora_proximo_evento()
            
            #Reviso si quedan camiones por atender
            if len(Entidad.orden_llegada) > 0:
                Camion = Entidad.orden_llegada.pop()
                if Camion.estado == "Cargando":
                    Entidad.disponibilidad = False
                    Entidad.horas_eventos["Carga"] = Reloj + Entidad.duracion_atencion()
                    Entidad.hora_proximo_evento()
                Entidad.orden_llegada.append(Camion)
            
            #Si hay fila la avanzo
            if len(Entidad.fila) > 0:
                Camion = Entidad.fila.popleft()
                Entidad.orden_llegada.append(Camion)
                i = Entidad.espacios_aculatamiento.index(0)
                Entidad.espacios_aculatamiento[i] = Camion
                Camion.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)  
        
        if Entidad.nombre_proximo_evento() == "Caida":
            Entidad.caida(Reloj) #Se actualiza la disponibilidad, el estado y se genera un tiempo de reparacion
            #Se corre el PL
            #######################DEFINIR ESTRUCTURA DEL OUTPUT DEL PL DE FORMA COMPATIBLE########################
            PF0, PM0, TVR, t_global, NC, L, Ttot, NP, NombrePala, LocPala, CP, TC, PDisp, Ley, ND, NombreDestino, TD, CD, DDisp, \
                cS1, cS4, cS2, cS3, cS5, NombreFases, F, DMPP, CEPP, alphaPM, alphaPF, alphaSlack, DifMaxPal = Inputs_PL(Reloj, total_camiones, size_camion, Palas, D_Entidades, Destinos, Nodos_Fase, L_Stocks, L_Botaderos, Plan_Esteril_Actualizado, Plan_Mineral_Actualizado)

            PL = process(TVR, alphaPM, alphaPF, alphaSlack, DifMaxPal, L, NombrePala, ND, NP, PM0, F, PF0, PDisp, TC, CP, TD, CD, DDisp, \
                DMPP, cS1, cS2, cS3, cS4, cS5, NC, t_global, LocPala, NombreDestino, Ley, LeyPromedio, NombreFases)
            #Debo identificar a todos los camiones a los cuales tengo que correr el Dinamico:
            viajando = camiones_viajando(D_Camiones, Entidad, D_Entidades)
            orden_asignacion = [trck for trck in Entidad.orden_llegada] + [trck for trck in Entidad.fila] + viajando
            #Salen los camiones de la fila y del espacio de aculatado
            Entidad.fila = deque()
            Entidad.orden_llegada = deque()
            Entidad.espacios_aculatamiento = [0 for trck in Entidad.espacios_aculatamiento]
            
            for camion in orden_asignacion:
                NI, NT = getNINT(camion, D_Entidades)
                nodo_inicio = nuevoNodoInicioTrasCaida(caminosMinimos, NI, NT, Reloj-camion.hora_salida)
                
                nuevo_destino = dinamico_dispatch(caminosMinimosYTiemposAcumulados, camion, camiones_asignables(D_Camiones, D_Entidades), D_Palas, MTV, Reloj, PL, D_Entidades, nodo_casino)   
                camion.OD = nuevo_destino #Guardo el resultado del dinamico - par OD
                TdV = MTV[nodo_inicio][camion.OD[0]]
                camion.redirigir_viaje(self, nodo_inicio, Reloj, TdV)
                
                
                D_Entidades[camion.destino].hora_ultima_asignacion = Reloj
                D_Entidades[camion.destino].camiones_asignados.append((Reloj, camion))
                
        if Entidad.nombre_proximo_evento() == "Recuperacion":
            #################################CORRER EL PL##############################
            Entidad.recuperacion(Reloj)
            PF0, PM0, TVR, t_global, NC, L, Ttot, NP, NombrePala, LocPala, CP, TC, PDisp, Ley, ND, NombreDestino, TD, CD, DDisp, \
                cS1, cS4, cS2, cS3, cS5, NombreFases, F, DMPP, CEPP, alphaPM, alphaPF, alphaSlack, DifMaxPal = Inputs_PL(Reloj, total_camiones, size_camion, Palas, D_Entidades, Destinos, Nodos_Fase, L_Stocks, L_Botaderos, Plan_Esteril_Actualizado, Plan_Mineral_Actualizado)

            PL = process(TVR, alphaPM, alphaPF, alphaSlack, DifMaxPal, L, NombrePala, ND, NP, PM0, F, PF0, PDisp, TC, CP, TD, CD, DDisp, \
                DMPP, cS1, cS2, cS3, cS4, cS5, NC, t_global, LocPala, NombreDestino, Ley, LeyPromedio, NombreFases)
            
            pass
            

        print("Nuevo estado de la pala {} a las {}".format(Entidad, Reloj))
        print("    Disponibilidad: {}".format(Entidad.disponibilidad))
        print("    Estado: {}".format(Entidad.estado))
        print("    Orden de llegada: {}".format(Entidad.orden_llegada))
        print("    Espacios aculatamiento: {}".format(Entidad.espacios_aculatamiento))
        print("    Fila: {}".format(Entidad.fila))
        print("    TPH Real: {}".format(Entidad.TPH))
        #print("    Toneladas sacadas: {}".format(Entidad.toneladas_sacadas))
        #print("    Camiones asignados: {}".format(Entidad.camiones_asignados))
        print("    Siguiente evento: {}".format(Entidad.hora_siguiente))
                 
############################################CHANCADOR#########################################################
    
    if Entidad.__class__.__name__ == "Chancador":
        
        if Entidad.disponibilidad: 
            Entidad.caida(Reloj)
            viajando = camiones_viajando(D_Camiones, Entidad, D_Entidades)
            PF0, PM0, TVR, t_global, NC, L, Ttot, NP, NombrePala, LocPala, CP, TC, PDisp, Ley, ND, NombreDestino, TD, CD, DDisp, \
                cS1, cS4, cS2, cS3, cS5, NombreFases, F, DMPP, CEPP, alphaPM, alphaPF, alphaSlack, DifMaxPal = Inputs_PL(Reloj, total_camiones, size_camion, Palas, D_Entidades, Destinos, Nodos_Fase, L_Stocks, L_Botaderos, Plan_Esteril_Actualizado, Plan_Mineral_Actualizado)

            PL = process(TVR, alphaPM, alphaPF, alphaSlack, DifMaxPal, L, NombrePala, ND, NP, PM0, F, PF0, PDisp, TC, CP, TD, CD, DDisp, \
                DMPP, cS1, cS2, cS3, cS4, cS5, NC, t_global, LocPala, NombreDestino, Ley, LeyPromedio, NombreFases)
            #Redirigir los camiones hacia el stock pile asignado al chancador
            for camion in viajando: #Los camiones que van viajando se van al stock pile (redirigen su viaje)
                NI, NT = getNINT(camion, D_Entidades)
                nodo_inicio = nuevoNodoInicioTrasCaida(caminosMinimos, NI, NT, Reloj-camion.hora_salida)
                camion.OD = (camion.OD[0], Entidad.stock_pile) 
                TdV = MTV[nodo_inicio][Entidad.stock_pile]
                camion.estado = "Viajando"
                camion.origen = camion.OD[0]
                camion.destino = Entidad.stock_pile
                camion.hora_siguiente = Reloj + TdV

            for camion in Entidad.fila: #Los camiones en la fila comienzan un viaje nuevo
                camion.OD = (Entidad.ubicacion, Entidad.stock_pile)
                TdV = MTV[Entidad.ubicacion][Entidad.stock_pile]
                camion.estado = "Viajando"
                camion.origen = camion.OD[0]
                camion.destino = Entidad.stock_pile
                camion.hora_salida = Reloj
                camion.hora_siguiente = Reloj + TdV

            Entidad.fila = [0 for trck in Entidad.fila]
            
            
        elif not Entidad.disponibilidad: #Si esque estaba funcionado y se gatillo un evento significa que se cayo
            Entidad.recuperacion(Reloj)
            ####CORRER PL
            PF0, PM0, TVR, t_global, NC, L, Ttot, NP, NombrePala, LocPala, CP, TC, PDisp, Ley, ND, NombreDestino, TD, CD, DDisp, \
                cS1, cS4, cS2, cS3, cS5, NombreFases, F, DMPP, CEPP, alphaPM, alphaPF, alphaSlack, DifMaxPal = Inputs_PL(Reloj, total_camiones, size_camion, Palas, D_Entidades, Destinos, Nodos_Fase, L_Stocks, L_Botaderos, Plan_Esteril_Actualizado, Plan_Mineral_Actualizado)

            PL = process(TVR, alphaPM, alphaPF, alphaSlack, DifMaxPal, L, NombrePala, ND, NP, PM0, F, PF0, PDisp, TC, CP, TD, CD, DDisp, \
                DMPP, cS1, cS2, cS3, cS4, cS5, NC, t_global, LocPala, NombreDestino, Ley, LeyPromedio, NombreFases)
            
    print('------------------------------------------------------------------------------------------') 
    L_Entidades.sort(key=lambda x: x.hora_siguiente, reverse=False)
TiempoSimulacion = time.time() - start_time
print("El tiempo de simulacion fue: %s seconds ---" % (time.time() - start_time))        

--- 0.0 seconds ---
Estado del Camion C28 a las 720.5977039945045: Aculatandose
    Origen: CS03
    Destino: PAB
    Hora de Salida: 700.3873345037329
    Estado: Aculatandose
    Almuerzo: 0
    Cargado: 0
    Carga: 0


Nuevo estado del Camion C28 a las 720.5977039945045: Cargando
    Origen: CS03
    Destino: PAB
    Hora de Salida: 700.3873345037329
    Estado: Cargando
    Almuerzo: 0
    Cargado: 0
    Carga: 0
    Siguiente evento: 71339.17269545594

------------------------------------------------------------------------------------------
Estado de la pala PAB a las 720.7538663352846
    Disponibilidad: False
    Estado: Funcionando
    Orden de llegada: deque([C32, C28])
    Espacios aculatamiento: [C32, C28]
    Fila: deque([])
    Capacidad: 303
Nuevo estado de la pala PAB a las 720.7538663352846
    Disponibilidad: False
    Estado: Funcionando
    Orden de llegada: deque([C28])
    Espacios aculatamiento: [0, C28]
    Fila: deque([])
    TPH Real: {}
    Siguiente evento:

    Carga: 0
    Siguiente evento: 73060.44368506593

------------------------------------------------------------------------------------------
Estado de la pala PAA a las 738.1955722845128
    Disponibilidad: False
    Estado: Funcionando
    Orden de llegada: deque([C2, C9])
    Espacios aculatamiento: [C9, C2]
    Fila: deque([C12])
    Capacidad: 303
Nuevo estado de la pala PAA a las 738.1955722845128
    Disponibilidad: False
    Estado: Funcionando
    Orden de llegada: deque([C9, C12])
    Espacios aculatamiento: [C9, C12]
    Fila: deque([])
    TPH Real: {}
    Siguiente evento: 739.2746738135783
------------------------------------------------------------------------------------------
Estado del Camion C35 a las 738.2121166737485: Viajando
    Origen: S2
    Destino: PAA
    Hora de Salida: 711.8339228085044
    Estado: Viajando
    Almuerzo: 0
    Cargado: 0
    Carga: 0


Nuevo estado del Camion C35 a las 738.2121166737485: En fila
    Origen: S2
    Destino: PAA
    Hora 

    Siguiente evento: 76035.13503086616

------------------------------------------------------------------------------------------
Estado del Camion C0 a las 768.0621190851032: Viajando
    Origen: PAB
    Destino: CS03
    Hora de Salida: 728.7839428218518
    Estado: Viajando
    Almuerzo: 0
    Cargado: 1
    Carga: 303


Nuevo estado del Camion C0 a las 768.0621190851032: En fila
    Origen: PAB
    Destino: CS03
    Hora de Salida: 728.7839428218518
    Estado: En fila
    Almuerzo: 0
    Cargado: 1
    Carga: 303
    Siguiente evento: 76038.14978942521

------------------------------------------------------------------------------------------
Estado del Camion C15 a las 768.1114247424192: Aculatandose
    Origen: S2
    Destino: PAB
    Hora de Salida: 738.680943105529
    Estado: Aculatandose
    Almuerzo: 0
    Cargado: 0
    Carga: 0


Nuevo estado del Camion C15 a las 768.1114247424192: Cargando
    Origen: S2
    Destino: PAB
    Hora de Salida: 738.680943105529
    Estado:

Estado del Camion C19 a las 775.0964832911907: Viajando
    Origen: PAB
    Destino: CS03
    Hora de Salida: 735.8183070279393
    Estado: Viajando
    Almuerzo: 0
    Cargado: 1
    Carga: 303


Nuevo estado del Camion C19 a las 775.0964832911907: En fila
    Origen: PAB
    Destino: CS03
    Hora de Salida: 735.8183070279393
    Estado: En fila
    Almuerzo: 0
    Cargado: 1
    Carga: 303
    Siguiente evento: 76734.55184582788

------------------------------------------------------------------------------------------
Estado del Camion C8 a las 775.2681304978046: Aculatandose
    Origen: PAC
    Destino: S2
    Hora de Salida: 716.6345515615219
    Estado: Aculatandose
    Almuerzo: 0
    Cargado: 1
    Carga: 293


Nuevo estado del Camion C8 a las 775.2681304978046: Descargando
    Origen: PAC
    Destino: S2
    Hora de Salida: 716.6345515615219
    Estado: Descargando
    Almuerzo: 0
    Cargado: 1
    Carga: 293
    Siguiente evento: 776.4347971644712

-------------------------

('PAC', 'S2')

Nuevo estado del Camion C6 a las 781.1989530627404: Viajando
    Origen: S2
    Destino: PAC
    Hora de Salida: 781.1989530627404
    Estado: Viajando
    Almuerzo: 0
    Cargado: 0
    Carga: 0
    Siguiente evento: 813.9202921432358

------------------------------------------------------------------------------------------
Estado del Camion C9 a las 781.6416686490256: Descargando
    Origen: PAA
    Destino: CS03
    Hora de Salida: 739.2746738135783
    Estado: Descargando
    Almuerzo: 0
    Cargado: 1
    Carga: 303

('PAA', 'CS03')

Nuevo estado del Camion C9 a las 781.6416686490256: Viajando
    Origen: CS03
    Destino: PAA
    Hora de Salida: 781.6416686490256
    Estado: Viajando
    Almuerzo: 0
    Cargado: 0
    Carga: 0
    Siguiente evento: 798.7912765926885

------------------------------------------------------------------------------------------
Estado de la pala PAB a las 781.7087711809396
    Disponibilidad: False
    Estado: Funcionando
    Orden de 

...

('PAA', 'CS03')

Nuevo estado del Camion C25 a las 1415.9430852207581: Viajando
    Origen: CS03
    Destino: PAA
    Hora de Salida: 1415.9430852207581
    Estado: Viajando
    Almuerzo: 2
    Cargado: 0
    Carga: 0
    Siguiente evento: 1433.092693164421

------------------------------------------------------------------------------------------
Estado del Camion C19 a las 1417.2346454926908: Viajando
    Origen: PAA
    Destino: CS03
    Hora de Salida: 1383.888559099451
    Estado: Viajando
    Almuerzo: 2
    Cargado: 1
    Carga: 303


Nuevo estado del Camion C19 a las 1417.2346454926908: Aculatandose
    Origen: PAA
    Destino: CS03
    Hora de Salida: 1383.888559099451
    Estado: Aculatandose
    Almuerzo: 2
    Cargado: 1
    Carga: 303
    Siguiente evento: 1418.4013121593575

------------------------------------------------------------------------------------------
Estado del Camion C36 a las 1418.1623606483845: Viajando
    Origen: PAB
    Destino: CS03
    Hora de Salida

('PAD', 'S2')

Nuevo estado del Camion C30 a las 1425.793613338089: Viajando
    Origen: CS03
    Destino: PAD
    Hora de Salida: 1425.793613338089
    Estado: Viajando
    Almuerzo: 2
    Cargado: 0
    Carga: 0
    Siguiente evento: 1448.980094243543

------------------------------------------------------------------------------------------
Estado del Camion C13 a las 1425.823595740707: Aculatandose
    Origen: CS03
    Destino: PAC
    Hora de Salida: 1401.1641759151262
    Estado: Aculatandose
    Almuerzo: 2
    Cargado: 0
    Carga: 0


Nuevo estado del Camion C13 a las 1425.823595740707: Cargando
    Origen: CS03
    Destino: PAC
    Hora de Salida: 1401.1641759151262
    Estado: Cargando
    Almuerzo: 2
    Cargado: 0
    Carga: 0
    Siguiente evento: 141156.53597833

------------------------------------------------------------------------------------------
Estado del Camion C17 a las 1425.8855974159048: Descargando
    Origen: PAC
    Destino: S2
    Hora de Salida: 1366.085

('PAD', 'S2')

Nuevo estado del Camion C3 a las 1428.3417490498123: Viajando
    Origen: S2
    Destino: PAD
    Hora de Salida: 1428.3417490498123
    Estado: Viajando
    Almuerzo: 2
    Cargado: 0
    Carga: 0
    Siguiente evento: 1460.7568158768474

------------------------------------------------------------------------------------------
Estado del Camion C27 a las 1428.3810980095068: Descargando
    Origen: PAD
    Destino: S2
    Hora de Salida: 1365.316698031035
    Estado: Descargando
    Almuerzo: 2
    Cargado: 1
    Carga: 293

('PAD', 'S2')

Nuevo estado del Camion C27 a las 1428.3810980095068: Viajando
    Origen: S2
    Destino: PAD
    Hora de Salida: 1428.3810980095068
    Estado: Viajando
    Almuerzo: 2
    Cargado: 0
    Carga: 0
    Siguiente evento: 1460.796164836542

------------------------------------------------------------------------------------------
Estado del Camion C24 a las 1428.5748032429622: Aculatandose
    Origen: PAC
    Destino: S2
    Hora de Sa

('PAB', 'CS03')

Nuevo estado del Camion C28 a las 1432.4470084131078: Viajando
    Origen: CS03
    Destino: PAB
    Hora de Salida: 1432.4470084131078
    Estado: Viajando
    Almuerzo: 2
    Cargado: 0
    Carga: 0
    Siguiente evento: 1451.4822374617502

------------------------------------------------------------------------------------------
Estado de la pala PAC a las 1432.4961199146653
    Disponibilidad: False
    Estado: Funcionando
    Orden de llegada: deque([C8, C15])
    Espacios aculatamiento: [C8, C15]
    Fila: deque([])
    Capacidad: 293
Nuevo estado de la pala PAC a las 1432.4961199146653
    Disponibilidad: True
    Estado: Funcionando
    Orden de llegada: deque([C15])
    Espacios aculatamiento: [0, C15]
    Fila: deque([])
    TPH Real: {}
    Siguiente evento: 141817.11587155185
------------------------------------------------------------------------------------------
Estado del Camion C15 a las 1432.7870148977356: Aculatandose
    Origen: CS03
    Destino: PA

('PAB', 'CS03')

Nuevo estado del Camion C6 a las 1436.3862665078188: Viajando
    Origen: S2
    Destino: PAB
    Hora de Salida: 1436.3862665078188
    Estado: Viajando
    Almuerzo: 2
    Cargado: 0
    Carga: 0
    Siguiente evento: 1464.6500814780425

------------------------------------------------------------------------------------------
Estado de la pala PAA a las 1436.4153624524722
    Disponibilidad: False
    Estado: Funcionando
    Orden de llegada: deque([C25])
    Espacios aculatamiento: [0, C25]
    Fila: deque([])
    Capacidad: 303
Nuevo estado de la pala PAA a las 1436.4153624524722
    Disponibilidad: True
    Estado: Funcionando
    Orden de llegada: deque([])
    Espacios aculatamiento: [0, 0]
    Fila: deque([])
    TPH Real: {}
    Siguiente evento: 142205.12088279476
------------------------------------------------------------------------------------------
Estado del Camion C19 a las 1437.2830209084898: Viajando
    Origen: CS03
    Destino: PAA
    Hora de Sal

In [20]:
print("El tiempo de simulacion fue: %s seconds ---" % (TiempoSimulacion)) 

El tiempo de simulacion fue: 8.20723557472229 seconds ---


In [21]:
print(Plan_Mineral)
print(Plan_Mineral_Actualizado)
print(D_Plan_Esteril)
print(Plan_Esteril_Actualizado)

{'PAA': 16113.785889051493, 'PAB': 17364.67613973196, 'PAC': 16231.40752746677, 'PAD': 16355.530631677593}
{'PAA': 0, 'PAB': 0, 'PAC': 16231.40752746677, 'PAD': 16355.530631677593}
{'Fase1': 45000, 'Fase2': 30000, 'Fase3': 0, 'Fase4': 0}
{'Fase1': 31815, 'Fase2': 18866, 'Fase3': 0, 'Fase4': 0}
